In [1]:
using GLMakie

In [2]:
using LinearAlgebra

struct Point3D
    x::Float64
    y::Float64
    z::Float64
end

struct Face
    p1::Point3D
    p2::Point3D
    p3::Point3D
end

function cross_product(p1::Point3D, p2::Point3D, p3::Point3D)
    # Vector from p1 to p2
    u = [p2.x - p1.x, p2.y - p1.y, p2.z - p1.z]
    # Vector from p1 to p3
    v = [p3.x - p1.x, p3.y - p1.y, p3.z - p1.z]
    # Cross product
    return [
        u[2]*v[3] - u[3]*v[2],
        u[3]*v[1] - u[1]*v[3],
        u[1]*v[2] - u[2]*v[1]
    ]
end

function is_point_above_plane(point::Point3D, face::Face)
    # Normal vector of the face
    normal = cross_product(face.p1, face.p2, face.p3)
    # Vector from face.p1 to point
    v = [point.x - face.p1.x, point.y - face.p1.y, point.z - face.p1.z]
    # Dot product to check if point is above the plane
    return dot(normal, v) > 0
end

function convex_hull_3d(points::Vector{Point3D})
    if length(points) < 4
        return Face[]
    end

    # Find the point with minimum z-coordinate
    min_z = minimum(p.z for p in points)
    start_point = points[argmin([p.z for p in points])]

    # Initialize hull
    hull = Face[]
    visited = Set{Tuple{Point3D,Point3D}}()
    candidates = copy(points)

    # Find first face
    # Pick two other points to form initial face
    p1 = start_point
    # Find p2: point not collinear with p1
    p2 = nothing
    for p in candidates
        if p != p1
            p2 = p
            break
        end
    end
    if p2 === nothing
        return Face[]
    end

    # Find p3: point not coplanar with p1, p2
    p3 = nothing
    for p in candidates
        if p != p1 && p != p2
            normal = cross_product(p1, p2, p)
            if norm(normal) > 1e-10  # Not collinear
                p3 = p
                break
            end
        end
    end
    if p3 === nothing
        return Face[]
    end

    # Create first face
    first_face = Face(p1, p2, p3)
    
    # Ensure normal points outward
    # Check if most points are below the plane
    above_count = sum(is_point_above_plane(p, first_face) for p in candidates)
    if above_count > length(candidates)/2
        first_face = Face(p1, p3, p2)  # Reverse orientation
    end

    # Initialize queue with edges of first face
    edges = [
        (first_face.p1, first_face.p2),
        (first_face.p2, first_face.p3),
        (first_face.p3, first_face.p1)
    ]
    push!(hull, first_face)

    while !isempty(edges)
        edge = popfirst!(edges)
        if edge in visited
            continue
        end
        push!(visited, edge)

        # Find the point that forms the next face with this edge
        best_point = nothing
        best_angle = -Inf
        edge_vec = [edge[2].x - edge[1].x, edge[2].y - edge[1].y, edge[2].z - edge[1].z]
        
        for p in candidates
            if p != edge[1] && p != edge[2]
                # Create temporary face
                temp_face = Face(edge[1], edge[2], p)
                normal = cross_product(edge[1], edge[2], p)
                if norm(normal) > 1e-10  # Not coplanar
                    # Calculate angle between planes
                    prev_normal = cross_product(hull[end].p1, hull[end].p2, hull[end].p3)
                    angle = dot(normal, prev_normal)/(norm(normal)*norm(prev_normal))
                    if angle > best_angle
                        best_angle = angle
                        best_point = p
                    end
                end
            end
        end

        if best_point !== nothing
            new_face = Face(edge[1], edge[2], best_point)
            # Ensure normal points outward
            above_count = sum(is_point_above_plane(p, new_face) for p in candidates)
            if above_count > length(candidates)/2
                new_face = Face(edge[1], best_point, edge[2])
            end
            
            push!(hull, new_face)
            
            # Add new edges to queue
            new_edges = [
                (new_face.p1, new_face.p2),
                (new_face.p2, new_face.p3),
                (new_face.p3, new_face.p1)
            ]
            
            for new_edge in new_edges
                if new_edge ∉ visited
                    push!(edges, new_edge)
                end
            end
        end
    end

    return hull
end

# Example usage
function main()
    points = [
        Point3D(0.0, 0.0, 0.0),
        Point3D(1.0, 0.0, 0.0),
        Point3D(0.0, 1.0, 0.0),
        Point3D(0.0, 0.0, 1.0),
        Point3D(1.0, 1.0, 1.0),
        Point3D(0.5, 0.5, 0.5)
    ]
    
    hull = convex_hull_3d(points)
    
    println("Convex Hull Faces:")
    for (i, face) in enumerate(hull)
        println("Face $i:")
        println("  P1: ($(face.p1.x), $(face.p1.y), $(face.p1.z))")
        println("  P2: ($(face.p2.x), $(face.p2.y), $(face.p2.z))")
        println("  P3: ($(face.p3.x), $(face.p3.y), $(face.p3.z))")
    end
end

main()

Convex Hull Faces:
Face 1:
  P1: (0.0, 0.0, 0.0)
  P2: (1.0, 0.0, 0.0)
  P3: (0.0, 1.0, 0.0)
Face 2:
  P1: (0.0, 0.0, 0.0)
  P2: (1.0, 0.0, 0.0)
  P3: (0.0, 1.0, 0.0)
Face 3:
  P1: (1.0, 0.0, 0.0)
  P2: (0.0, 1.0, 0.0)
  P3: (0.0, 0.0, 0.0)
Face 4:
  P1: (0.0, 1.0, 0.0)
  P2: (0.0, 0.0, 0.0)
  P3: (1.0, 0.0, 0.0)


In [5]:
typeof(convex_hull_3d([Point3D(0, 0, 0), Point3D(1, 0, 0), Point3D(0, 1, 0), Point3D(0, 0, 1)]))

Vector{Face} (alias for Array{Face, 1})

In [25]:
GLMakie.GeometryBasics.mesh(hull::Vector{Face}) = GLMakie.GeometryBasics.Mesh(
    vertices = [Point3{Float64}(p.x, p.y, p.z) for face in hull for p in [face.p1, face.p2, face.p3]],
    faces = [[i, i+1, i+2] for i in 1:3:length(hull)*3]
)

In [29]:
vertices = [Point3f(0, 0, 0), Point3f(1, 0, 0), Point3f(0, 1, 0), Point3f(0, 0, 1)]
faces = [
    1 2 3
    2 3 4
    ]  # Note the space-separated matrix

mesh(vertices, faces)

In [33]:
A = [[i, i+1, i+2] for i in 1:3:4*3]

4-element Vector{Vector{Int64}}:
 [1, 2, 3]
 [4, 5, 6]
 [7, 8, 9]
 [10, 11, 12]

In [1]:
import GLMakie
function GLMakie.GeometryBasics.Mesh(hull::Vector{Face})
    vertices = [Point3f(p.x, p.y, p.z) for face in hull for p in [face.p1, face.p2, face.p3]]
    faces = [[i, i+1, i+2] for i in 1:3:length(hull)*3]
    faces = GLMakie.GeometryBasics.NgonFace{3, Int}[faces...]
    return GLMakie.GeometryBasics.Mesh(vertices, faces)
end

UndefVarError: UndefVarError: `Face` not defined in `Main`
Suggestion: check for spelling errors or missing imports.
Hint: a global variable of this name may be made accessible by importing StyledStrings in the current active module Main

In [67]:
M = GLMakie.GeometryBasics.Mesh(convex_hull_3d([Point3D(0, 0, 0), Point3D(1, 0, 0), Point3D(0, 1, 0), Point3D(0, 0, 1)]))

Mesh{3, Float32, GeometryBasics.TriangleFace{Int64}}
    faces: 4
    vertex position: 12


In [69]:
mesh(M)

In [2]:
GLMakie.GeometryBasics.NgonFace{3, Int}[
    [1 2 3],
    [4 5 6]
]

2-element Vector{GeometryBasics.TriangleFace{Int64}}:
 TriangleFace{Int64}(1, 2, 3)
 TriangleFace{Int64}(4, 5, 6)

In [3]:
A = [[1,2,3], [4,5,6]]
GLMakie.GeometryBasics.NgonFace{3, Int}[A...]

2-element Vector{GeometryBasics.TriangleFace{Int64}}:
 TriangleFace{Int64}(1, 2, 3)
 TriangleFace{Int64}(4, 5, 6)

In [4]:
B = [A...]

2-element Vector{Vector{Int64}}:
 [1, 2, 3]
 [4, 5, 6]

In [36]:
using GLMakie
import GLMakie.GeometryBasics

# Example of a mesh
S = GeometryBasics.mesh(GeometryBasics.Sphere(Point3(0.0, 0.0, 0.0), 1.0))

S_vertices = S.vertex_attributes[1]  #
# 576-element Vector{Point{3, Float64}}:
#  [0.0, 0.0, 1.0]
#  [0.1361666490962466, 0.0, 0.9906859460363308]
#  [0.2697967711570243, 0.0, 0.9629172873477994]
#  .
#  .
#  .


S_faces = S.faces  
# 1058-element Vector{GeometryBasics.NgonFace{3, GeometryBasics.OffsetInteger{-1, UInt32}}}:
#  GLTriangleFace(1, 2, 26)
#  GLTriangleFace(1, 26, 25)
#  .
#  .
#  .

[Int(S_faces[1][index].i) for index in 1:length(S_faces[1])]
# 3-element Vector{Int64}:
#   0
#   1
#  25


# # Example of how you would create your own mesh
vertices = [Point3f(0, 0, 0), Point3f(1, 0, 0), Point3f(0, 1, 0), Point3f(0, 0, 1)]
faces = [[1, 2, 3], [2, 3, 4]]  # Note the space-separated matrix
faces = GLMakie.GeometryBasics.NgonFace{3, Int}[faces...]
GLMakie.GeometryBasics.Mesh(vertices, faces)



Mesh{3, Float32, GeometryBasics.TriangleFace{Int64}}
    faces: 2
    vertex position: 4


In [26]:
fieldnames(GLMakie.GLIndex)

(:i,)

In [37]:
using GLMakie
import GLMakie.GeometryBasics
using LinearAlgebra
using DataStructures  # For priority queue

In [46]:
using GLMakie
import GLMakie.GeometryBasics
using LinearAlgebra
using DataStructures  # For priority queue
using Statistics: mean

# Compute triangle quality (aspect ratio: longest edge / shortest edge)
function compute_triangle_quality(vertices::Vector{Point3f}, face::GeometryBasics.NgonFace{3, Int})
    v0 = vertices[face[1]]
    v1 = vertices[face[2]]
    v2 = vertices[face[3]]
    edges = [
        norm(v1 - v0),
        norm(v2 - v1),
        norm(v0 - v2)
    ]
    shortest = minimum(edges)
    longest = maximum(edges)
    return shortest > 0 ? longest / shortest : Inf
end

# Compute cost of collapsing an edge, balancing geometric error and triangle quality
function compute_collapse_cost(vertices::Vector{Point3f}, faces::Vector{GeometryBasics.NgonFace{3, Int}}, edge::Tuple{Int, Int})
    v1, v2 = edge
    v_new = (vertices[v1] + vertices[v2]) / 2  # Midpoint (could optimize position)
    
    # Find affected faces (those containing v1 or v2)
    affected_faces = [f for f in faces if v1 in f || v2 in f]
    
    # Geometric error (distance between vertices)
    error = norm(vertices[v1] - vertices[v2])
    
    # Simulate collapse and compute average aspect ratio of resulting triangles
    temp_vertices = copy(vertices)
    temp_vertices[v1] = v_new
    temp_vertices[v2] = v_new
    aspect_ratios = Float64[]
    for face in affected_faces
        new_face = [idx == v1 || idx == v2 ? v1 : idx for idx in face]
        if length(unique(new_face)) == 3  # Non-degenerate triangle
            push!(aspect_ratios, compute_triangle_quality(temp_vertices, GeometryBasics.NgonFace{3, Int}(new_face...)))
        end
    end
    
    # Penalize poor triangle quality
    avg_aspect_ratio = isempty(aspect_ratios) ? 1.0 : mean(aspect_ratios)
    cost = error + 5.0 * max(0, avg_aspect_ratio - 1.0)  # Weight quality penalty
    return cost
end

# Extract unique edges from faces
function get_edges(faces::Vector{GeometryBasics.NgonFace{3, Int}})
    edges = Set{Tuple{Int, Int}}()
    for face in faces
        v1, v2, v3 = face
        push!(edges, tuple(sort([v1, v2])...))
        push!(edges, tuple(sort([v2, v3])...))
        push!(edges, tuple(sort([v1, v3])...))
    end
    return collect(edges)
end

# Simplify mesh while optimizing for well-sized triangles
function simplify_mesh_well_sized(vertices::Vector{Point3f}, faces::Vector{GeometryBasics.NgonFace{3, Int}}, target_vertex_count::Int)
    vertices = copy(vertices)
    faces = copy(faces)
    
    # Initialize priority queue with edge costs
    edges = get_edges(faces)
    edge_queue = PriorityQueue{Tuple{Int, Int}, Float64}()
    for edge in edges
        enqueue!(edge_queue, edge, compute_collapse_cost(vertices, faces, edge))
    end
    
    # Track vertex mapping - use Dict for cleaner mapping
    vertex_map = Dict{Int, Int}()
    for i in 1:length(vertices)
        vertex_map[i] = i
    end
    
    # Collapse edges until target is reached
    while length(unique(values(vertex_map))) > target_vertex_count && !isempty(edge_queue)
        edge, cost = dequeue_pair!(edge_queue)
        v1, v2 = edge
        
        # Skip if vertices already merged
        if vertex_map[v1] == vertex_map[v2]
            continue
        end
        
        # Create new vertex at midpoint
        v_new = (vertices[v1] + vertices[v2]) / 2
        push!(vertices, v_new)
        new_idx = length(vertices)
        
        # Update vertex mapping - map both vertices to new index
        old_v1_idx = vertex_map[v1]
        old_v2_idx = vertex_map[v2]
        
        # Update all vertices that were mapped to either v1 or v2
        for (k, v) in vertex_map
            if v == old_v1_idx || v == old_v2_idx
                vertex_map[k] = new_idx
            end
        end
        
        # Add mapping for the new vertex
        vertex_map[new_idx] = new_idx
        
        # Update faces with new mapping
        new_faces = GeometryBasics.NgonFace{3, Int}[]
        for face in faces
            new_face = [vertex_map[idx] for idx in face]
            if length(unique(new_face)) == 3  # Keep non-degenerate faces
                push!(new_faces, GeometryBasics.NgonFace{3, Int}(new_face...))
            end
        end
        faces = new_faces
        
        # Update edge list and recompute costs
        edges = get_edges(faces)
        edge_queue = PriorityQueue{Tuple{Int, Int}, Float64}()
        for edge in edges
            if haskey(vertex_map, edge[1]) && haskey(vertex_map, edge[2])
                enqueue!(edge_queue, edge, compute_collapse_cost(vertices, faces, edge))
            end
        end
    end
    
    # Clean up vertices and reindex faces
    used_vertices = unique([idx for face in faces for idx in face])
    sort!(used_vertices)
    
    # Create final vertex mapping
    final_vertex_map = Dict{Int, Int}()
    for (new_idx, old_idx) in enumerate(used_vertices)
        final_vertex_map[old_idx] = new_idx
    end
    
    # Create final vertices and faces
    final_vertices = vertices[used_vertices]
    final_faces = [GeometryBasics.NgonFace{3, Int}([final_vertex_map[idx] for idx in face]...) for face in faces]
    
    return final_vertices, final_faces
end

# Example: Simplify sphere mesh and visualize
function main()
    # Create sphere mesh
    S = GeometryBasics.mesh(GeometryBasics.Sphere(Point3f(0.0, 0.0, 0.0), 1.0))
    vertices = S.position  # Vector{Point3f}
    faces = S.faces       # Vector{NgonFace{3, OffsetInteger{-1, UInt32}}}
    
    # Convert faces to use 1-based indexing
    faces = [GeometryBasics.NgonFace{3, Int}([Int(f[i].i) + 1 for i in 1:3]...) for f in faces]
    
    # Simplify to 50% of original vertices
    target_vertices = Int(round(length(vertices) * 0.5))
    new_vertices, new_faces = simplify_mesh_well_sized(vertices, faces, target_vertices)
    
    # Create new mesh
    new_mesh = GeometryBasics.Mesh(new_vertices, new_faces)
    
    # Visualize original and simplified meshes
    fig = Figure()
    
    # Original mesh
    ax1 = Axis3(fig[1, 1], title="Original Mesh")
    mesh!(ax1, S, color=:blue)
    
    # Simplified mesh
    ax2 = Axis3(fig[1, 2], title="Simplified Mesh")
    mesh!(ax2, new_mesh, color=:red)
    
    display(fig)
    
    return new_mesh
end

Edit = main()

Mesh{3, Float32, GeometryBasics.TriangleFace{Int64}}
    faces: 504
    vertex position: 288


In [47]:
mesh(Edit)

In [38]:
using GLMakie
import GLMakie.GeometryBasics
using LinearAlgebra
using DataStructures  # For priority queue
using Statistics: mean

# Compute triangle quality (aspect ratio: longest edge / shortest edge)
function compute_triangle_quality(vertices::Vector{Point3f}, face::GeometryBasics.NgonFace{3, Int})
    v0 = vertices[face[1]]
    v1 = vertices[face[2]]
    v2 = vertices[face[3]]
    edges = [
        norm(v1 - v0),
        norm(v2 - v1),
        norm(v0 - v2)
    ]
    shortest = minimum(edges)
    longest = maximum(edges)
    return shortest > 0 ? longest / shortest : Inf
end

# Compute cost of collapsing an edge, balancing geometric error and triangle quality
function compute_collapse_cost(vertices::Vector{Point3f}, faces::Vector{GeometryBasics.NgonFace{3, Int}}, edge::Tuple{Int, Int})
    v1, v2 = edge
    v_new = (vertices[v1] + vertices[v2]) / 2  # Midpoint (could optimize position)
    
    # Find affected faces (those containing v1 or v2)
    affected_faces = [f for f in faces if v1 in f || v2 in f]
    
    # Geometric error (distance between vertices)
    error = norm(vertices[v1] - vertices[v2])
    
    # Simulate collapse and compute average aspect ratio of resulting triangles
    temp_vertices = copy(vertices)
    temp_vertices[v1] = v_new
    temp_vertices[v2] = v_new
    aspect_ratios = Float64[]
    for face in affected_faces
        new_face = [idx == v1 || idx == v2 ? v1 : idx for idx in face]
        if length(unique(new_face)) == 3  # Non-degenerate triangle
            push!(aspect_ratios, compute_triangle_quality(temp_vertices, GeometryBasics.NgonFace{3, Int}(new_face...)))
        end
    end
    
    # Penalize poor triangle quality
    avg_aspect_ratio = isempty(aspect_ratios) ? 1.0 : mean(aspect_ratios)
    cost = error + 5.0 * max(0, avg_aspect_ratio - 1.0)  # Weight quality penalty
    return cost
end

# Extract unique edges from faces
function get_edges(faces::Vector{GeometryBasics.NgonFace{3, Int}})
    edges = Set{Tuple{Int, Int}}()
    for face in faces
        v1, v2, v3 = face
        push!(edges, tuple(sort([v1, v2])...))
        push!(edges, tuple(sort([v2, v3])...))
        push!(edges, tuple(sort([v1, v3])...))
    end
    return collect(edges)
end

# Simplify mesh while optimizing for well-sized triangles
function simplify_mesh_well_sized(vertices::Vector{Point3f}, faces::Vector{GeometryBasics.NgonFace{3, Int}}, target_vertex_count::Int)
    vertices = copy(vertices)
    faces = copy(faces)
    
    # Initialize priority queue with edge costs
    edges = get_edges(faces)
    edge_queue = PriorityQueue{Tuple{Int, Int}, Float64}()
    for edge in edges
        enqueue!(edge_queue, edge, compute_collapse_cost(vertices, faces, edge))
    end
    
    # Track vertex mapping - use Dict for cleaner mapping
    vertex_map = Dict{Int, Int}()
    for i in 1:length(vertices)
        vertex_map[i] = i
    end
    
    # Collapse edges until target is reached
    while length(unique(values(vertex_map))) > target_vertex_count && !isempty(edge_queue)
        edge, cost = dequeue_pair!(edge_queue)
        v1, v2 = edge
        
        # Skip if vertices already merged
        if vertex_map[v1] == vertex_map[v2]
            continue
        end
        
        # Create new vertex at midpoint
        v_new = (vertices[v1] + vertices[v2]) / 2
        push!(vertices, v_new)
        new_idx = length(vertices)
        
        # Update vertex mapping - map both vertices to new index
        old_v2_idx = vertex_map[v2]
        for (k, v) in vertex_map
            if v == vertex_map[v1] || v == old_v2_idx
                vertex_map[k] = new_idx
            end
        end
        
        # Update faces with new mapping
        new_faces = GeometryBasics.NgonFace{3, Int}[]
        for face in faces
            new_face = [vertex_map[idx] for idx in face]
            if length(unique(new_face)) == 3  # Keep non-degenerate faces
                push!(new_faces, GeometryBasics.NgonFace{3, Int}(new_face...))
            end
        end
        faces = new_faces
        
        # Update edge list and recompute costs
        edges = get_edges(faces)
        edge_queue = PriorityQueue{Tuple{Int, Int}, Float64}()
        for edge in edges
            if haskey(vertex_map, edge[1]) && haskey(vertex_map, edge[2])
                enqueue!(edge_queue, edge, compute_collapse_cost(vertices, faces, edge))
            end
        end
    end
    
    # Clean up vertices and reindex faces
    used_vertices = unique([idx for face in faces for idx in face])
    sort!(used_vertices)
    
    # Create final vertex mapping
    final_vertex_map = Dict{Int, Int}()
    for (new_idx, old_idx) in enumerate(used_vertices)
        final_vertex_map[old_idx] = new_idx
    end
    
    # Create final vertices and faces
    final_vertices = vertices[used_vertices]
    final_faces = [GeometryBasics.NgonFace{3, Int}([final_vertex_map[idx] for idx in face]...) for face in faces]
    
    return final_vertices, final_faces
end

# Example: Simplify sphere mesh and visualize
function main()
    # Create sphere mesh
    S = GeometryBasics.mesh(GeometryBasics.Sphere(Point3f(0.0, 0.0, 0.0), 1.0))
    vertices = S.position  # Vector{Point3f}
    faces = S.faces       # Vector{NgonFace{3, OffsetInteger{-1, UInt32}}}
    
    # Convert faces to use 1-based indexing
    faces = [GeometryBasics.NgonFace{3, Int}([Int(f[i]) + 1 for i in 1:3]...) for f in faces]
    
    # Simplify to 50% of original vertices
    target_vertices = Int(round(length(vertices) * 0.5))
    new_vertices, new_faces = simplify_mesh_well_sized(vertices, faces, target_vertices)
    
    # Create new mesh
    new_mesh = GeometryBasics.Mesh(new_vertices, new_faces)
    
    # Visualize original and simplified meshes
    fig = Figure()
    
    # Original mesh
    ax1 = Axis3(fig[1, 1], title="Original Mesh ($(length(vertices)) vertices)")
    mesh!(ax1, S, color=:blue, alpha=0.8)
    
    # Simplified mesh
    ax2 = Axis3(fig[1, 2], title="Simplified Mesh ($(length(new_vertices)) vertices)")
    mesh!(ax2, new_mesh, color=:red, alpha=0.8)
    
    display(fig)
    
    return new_mesh
end

main()

ArgumentError: ArgumentError: Package DataStructures not found in current path.
- Run `import Pkg; Pkg.add("DataStructures")` to install the DataStructures package.

In [1]:
using Pkg
Pkg.status()

Status `C:\Users\aryan\.julia\environments\testing\Project.toml`
  [a93c6f00] DataFrames v1.7.0
  [07eb4e4e] Delaunay v1.2.0
  [927a84f5] DelaunayTriangulation v1.6.4
⌃ [ffbed154] DocStringExtensions v0.9.4
⌃ [e9467ef8] GLMakie v0.13.3
  [7073ff75] IJulia v1.29.0
  [682c06a0] JSON v0.21.4
Info Packages marked with ⌃ have new versions available and may be upgradable.


In [2]:
Pkg.add("Meshes")

   Resolving package versions...
   Installed ScopedValues ───────────── v1.4.0
   Installed ZygoteRules ────────────── v0.2.7
   Installed TiledIteration ─────────── v0.5.0
   Installed Distances ──────────────── v0.10.12
   Installed Static ─────────────────── v1.2.0
   Installed IRTools ────────────────── v0.4.15
   Installed NearestNeighbors ───────── v0.4.22
   Installed Bessels ────────────────── v0.2.8
   Installed IfElse ─────────────────── v0.1.1
   Installed CommonSubexpressions ───── v0.3.1
   Installed StaticArrayInterface ───── v1.8.0
   Installed CommonWorldInvalidations ─ v1.0.0
   Installed DiffRules ──────────────── v1.15.1
   Installed ChainRules ─────────────── v1.72.5
   Installed Zygote ─────────────────── v0.7.10
   Installed DiffResults ────────────── v1.1.0
   Installed Colorfy ────────────────── v1.1.0
   Installed ArrayInterface ─────────── v7.19.0
   Installed Rotations ──────────────── v1.7.1
   Installed SparseInverseSubset ────── v0.1.2
   Installed Forwar

In [8]:
using Meshes
import GLMakie.GeometryBasics
using GLMakie

In [9]:
S = GeometryBasics.mesh(GeometryBasics.Sphere(Point3f(0.0, 0.0, 0.0), 1.0))

Mesh{3, Float32, GeometryBasics.NgonFace{3, GeometryBasics.OffsetInteger{-1, UInt32}}}
    faces: 1058
    vertex position: 576


In [10]:
simp1 = simplify(S, SelingerSimplification(0.01))

MethodError: MethodError: no method matching simplify(::GeometryBasics.Mesh{3, Float32, GeometryBasics.NgonFace{3, GeometryBasics.OffsetInteger{-1, UInt32}}, (:position,), Tuple{Vector{GeometryBasics.Point{3, Float32}}}, Vector{GeometryBasics.NgonFace{3, GeometryBasics.OffsetInteger{-1, UInt32}}}}, ::SelingerSimplification{Unitful.Quantity{Float64, 𝐋, Unitful.FreeUnits{(m,), 𝐋, nothing}}})
The function `simplify` exists, but no method is defined for this combination of argument types.

Closest candidates are:
  simplify(!Matched::Chain, ::SelingerSimplification)
   @ Meshes C:\Users\aryan\.julia\packages\Meshes\oyDEM\src\simplification\selinger.jl:25
  simplify(!Matched::Domain, ::SimplificationMethod)
   @ Meshes C:\Users\aryan\.julia\packages\Meshes\oyDEM\src\simplification.jl:25
  simplify(!Matched::Multi, ::SimplificationMethod)
   @ Meshes C:\Users\aryan\.julia\packages\Meshes\oyDEM\src\simplification.jl:23
  ...


In [13]:
S = Meshes.SimpleMesh(S)

MethodError: MethodError: no method matching SimpleMesh(::GeometryBasics.Mesh{3, Float32, GeometryBasics.NgonFace{3, GeometryBasics.OffsetInteger{-1, UInt32}}, (:position,), Tuple{Vector{GeometryBasics.Point{3, Float32}}}, Vector{GeometryBasics.NgonFace{3, GeometryBasics.OffsetInteger{-1, UInt32}}}})
The type `SimpleMesh` exists, but no method is defined for this combination of argument types when trying to construct it.

Closest candidates are:
  SimpleMesh(::Any, !Matched::AbstractVector{<:Connectivity}; relations)
   @ Meshes C:\Users\aryan\.julia\packages\Meshes\oyDEM\src\domains\meshes\simplemesh.jl:41
  SimpleMesh(!Matched::V, !Matched::TP) where {M<:Meshes.Manifold, C<:CoordRefSystems.CRS, V<:AbstractArray{Meshes.Point{M, C}, 1}, TP<:Topology}
   @ Meshes C:\Users\aryan\.julia\packages\Meshes\oyDEM\src\domains\meshes\simplemesh.jl:35
  SimpleMesh(!Matched::AbstractVector{<:Tuple}, !Matched::Topology)
   @ Meshes C:\Users\aryan\.julia\packages\Meshes\oyDEM\src\domains\meshes\simp

In [15]:
using GLMakie
import GLMakie.GeometryBasics
using Meshes

# Create the original GeometryBasics mesh
S = GeometryBasics.mesh(GeometryBasics.Sphere(Point3f(0.0, 0.0, 0.0), 1.0))

# Convert GeometryBasics.Mesh to Meshes.SimpleMesh
function convert_to_simple_mesh(geom_mesh::GeometryBasics.Mesh)
    # Extract vertices - convert Point3f to tuples
    vertices = S.position  # This gets the vertices directly
    points = [(v[1], v[2], v[3]) for v in vertices]
    
    # Extract faces and convert to connectivity
    faces = S.faces
    # Convert from 0-based OffsetInteger to 1-based indices and create triangular connectivity
    connec = [connect((Int(f[1].i) + 1, Int(f[2].i) + 1, Int(f[3].i) + 1), Triangle) for f in faces]
    
    # Create SimpleMesh
    return SimpleMesh(points, connec)
end

# Convert the sphere mesh
simple_mesh = convert_to_simple_mesh(S)

# Verify the conversion
println("Original mesh vertices: ", length(S.position))
println("Original mesh faces: ", length(S.faces))
println("Simple mesh vertices: ", length(vertices(simple_mesh)))
println("Simple mesh elements: ", length(elements(simple_mesh)))

Original mesh vertices: 576
Original mesh faces: 1058
Simple mesh vertices: 576
Simple mesh elements: 1058


In [39]:
# Convert Meshes.SimpleMesh back to GeometryBasics.Mesh
function convert_to_makie_mesh(simple_mesh::SimpleMesh)
    # Extract vertices from SimpleMesh
    mesh_vertices = vertices(simple_mesh)
    points = [Point3f(v[1], v[2], v[3]) for v in mesh_vertices]
    
    # Extract faces from SimpleMesh
    mesh_elements = elements(simple_mesh)
    faces = GeometryBasics.NgonFace{3, Int}[]
    
    for element in mesh_elements
        # Get the vertex indices for this triangle
        indices = [v for v in element]
        push!(faces, GeometryBasics.NgonFace{3, Int}(indices...))
    end
    
    # Create GeometryBasics.Mesh
    return GeometryBasics.Mesh(points, faces)
end

# Test the conversion
makie_mesh = convert_to_makie_mesh(simple_mesh)

# Verify the conversion
println("Simple mesh vertices: ", length(vertices(simple_mesh)))
println("Simple mesh elements: ", length(elements(simple_mesh)))
println("Makie mesh vertices: ", length(makie_mesh.position))
println("Makie mesh faces: ", length(makie_mesh.faces))

# Visualize to confirm it works
mesh(makie_mesh)

MethodError: MethodError: no method matching getindex(::Meshes.Point{𝔼{3}, CoordRefSystems.Cartesian3D{CoordRefSystems.NoDatum, Unitful.Quantity{Float32, 𝐋, Unitful.FreeUnits{(m,), 𝐋, nothing}}}}, ::Int64)
The function `getindex` exists, but no method is defined for this combination of argument types.

In [20]:
viz(simple_mesh, showsegments = true)

┌ Warning: `shading = FastShading` is deprecated in favor of `shading = true` as a plot attribute. To switch between `FastShading` and `MultiLightShading` explicitly, use scene attributes.
└ @ Makie C:\Users\aryan\.julia\packages\Makie\aJUtI\src\conversions.jl:2343


In [36]:
simp1 = simplify(simple_mesh, SelingerSimplification(0.5))

1058 GeometrySet
├─ PolyArea((x: 0.136167 m, y: 0.0 m, z: 0.990686 m), (x: 0.131117 m, y: 0.0367373 m, z: 0.990686 m), (x: 0.0 m, y: 0.0 m, z: 1.0 m))
├─ PolyArea((x: 0.131117 m, y: 0.0367373 m, z: 0.990686 m), (x: 0.0 m, y: 0.0 m, z: 1.0 m), (x: 0.0 m, y: 0.0 m, z: 1.0 m))
├─ PolyArea((x: 0.269797 m, y: 0.0 m, z: 0.962917 m), (x: 0.259792 m, y: 0.0727903 m, z: 0.962917 m), (x: 0.136167 m, y: 0.0 m, z: 0.990686 m))
├─ PolyArea((x: 0.259792 m, y: 0.0727903 m, z: 0.962917 m), (x: 0.131117 m, y: 0.0367373 m, z: 0.990686 m), (x: 0.136167 m, y: 0.0 m, z: 0.990686 m))
├─ PolyArea((x: 0.398401 m, y: 0.0 m, z: 0.917211 m), (x: 0.383627 m, y: 0.107487 m, z: 0.917211 m), (x: 0.269797 m, y: 0.0 m, z: 0.962917 m))
⋮
├─ PolyArea((x: 0.269797 m, y: -6.60811f-17 m, z: -0.962917 m), (x: 0.398401 m, y: -9.75801f-17 m, z: -0.917211 m), (x: 0.383627 m, y: -0.107487 m, z: -0.917211 m))
├─ PolyArea((x: 0.131117 m, y: -0.0367373 m, z: -0.990686 m), (x: 0.136167 m, y: -3.33512f-17 m, z: -0.990686 m), (x: 0.2

In [37]:
viz(simp1, showsegments = true)

┌ Warning: `shading = FastShading` is deprecated in favor of `shading = true` as a plot attribute. To switch between `FastShading` and `MultiLightShading` explicitly, use scene attributes.
└ @ Makie C:\Users\aryan\.julia\packages\Makie\aJUtI\src\conversions.jl:2343


In [6]:
using GLMakie
import GLMakie.GeometryBasics
using LinearAlgebra

# Vertex clustering mesh simplification
function vertex_clustering_simplification(vertices::Vector{Point3f}, faces::Vector{GeometryBasics.NgonFace{3, Int}}, grid_size::Float64)
    # Create a spatial grid for clustering
    # Find bounding box
    min_coords = Point3f(Inf, Inf, Inf)
    max_coords = Point3f(-Inf, -Inf, -Inf)
    
    for v in vertices
        min_coords = Point3f(min(min_coords[1], v[1]), min(min_coords[2], v[2]), min(min_coords[3], v[3]))
        max_coords = Point3f(max(max_coords[1], v[1]), max(max_coords[2], v[2]), max(max_coords[3], v[3]))
    end
    
    # Create grid cells
    grid_cells = Dict{Tuple{Int, Int, Int}, Vector{Int}}()
    
    # Assign vertices to grid cells
    for (i, v) in enumerate(vertices)
        # Calculate grid cell coordinates
        cell_x = Int(floor((v[1] - min_coords[1]) / grid_size))
        cell_y = Int(floor((v[2] - min_coords[2]) / grid_size))
        cell_z = Int(floor((v[3] - min_coords[3]) / grid_size))
        
        cell_key = (cell_x, cell_y, cell_z)
        
        if !haskey(grid_cells, cell_key)
            grid_cells[cell_key] = Int[]
        end
        push!(grid_cells[cell_key], i)
    end
    
    # Create clustered vertices and mapping
    clustered_vertices = Point3f[]
    vertex_mapping = Dict{Int, Int}()
    
    for (cell_key, vertex_indices) in grid_cells
        if !isempty(vertex_indices)
            # Compute centroid of vertices in this cell
            centroid = Point3f(0.0, 0.0, 0.0)
            for idx in vertex_indices
                centroid = centroid + vertices[idx]
            end
            centroid = centroid / length(vertex_indices)
            
            # Add clustered vertex
            push!(clustered_vertices, centroid)
            cluster_idx = length(clustered_vertices)
            
            # Map all original vertices in this cell to the clustered vertex
            for idx in vertex_indices
                vertex_mapping[idx] = cluster_idx
            end
        end
    end
    
    # Update faces with new vertex indices
    new_faces = GeometryBasics.NgonFace{3, Int}[]
    for face in faces
        new_face_indices = [vertex_mapping[face[i]] for i in 1:3]
        
        # Only keep non-degenerate faces (all vertices different)
        if length(unique(new_face_indices)) == 3
            push!(new_faces, GeometryBasics.NgonFace{3, Int}(new_face_indices...))
        end
    end
    
    return clustered_vertices, new_faces
end

# Example usage
function demo_vertex_clustering()
    # Create original sphere mesh
    S = GeometryBasics.mesh(GeometryBasics.Sphere(Point3f(0.0, 0.0, 0.0), 1.0))
    vertices = S.position
    faces = [GeometryBasics.NgonFace{3, Int}([Int(f[i].i) + 1 for i in 1:3]...) for f in S.faces]
    
    println("Original mesh: $(length(vertices)) vertices, $(length(faces)) faces")
    
    # Apply vertex clustering with different grid sizes
    grid_sizes = [0.1, 0.2, 0.3, 0.5]
    
    fig = Figure(size = (1200, 800))
    
    # Original mesh
    ax1 = Axis3(fig[1, 1], title="Original Mesh\n$(length(vertices)) vertices")
    mesh!(ax1, S, color=:blue, alpha=0.8)
    
    for (i, grid_size) in enumerate(grid_sizes)
        clustered_vertices, clustered_faces = vertex_clustering_simplification(vertices, faces, grid_size)
        
        # Create new mesh
        clustered_mesh = GeometryBasics.Mesh(clustered_vertices, clustered_faces)
        
        # Calculate reduction percentage
        reduction = round((1 - length(clustered_vertices) / length(vertices)) * 100, digits=1)
        
        println("Grid size $grid_size: $(length(clustered_vertices)) vertices, $(length(clustered_faces)) faces ($(reduction)% reduction)")
        
        # Plot
        row = (i - 1) ÷ 2 + 1
        col = (i - 1) % 2 + 2
        ax = Axis3(fig[row, col], title="Grid Size: $grid_size\n$(length(clustered_vertices)) vertices ($(reduction)% reduction)")
        mesh!(ax, clustered_mesh, color=:red, alpha=0.8)
    end
    
    display(fig)
    
    # Return the most simplified mesh for further use
    final_vertices, final_faces = vertex_clustering_simplification(vertices, faces, 0.3)
    return GeometryBasics.Mesh(final_vertices, final_faces)
end

# Advanced vertex clustering with quality preservation
function adaptive_vertex_clustering(vertices::Vector{Point3f}, faces::Vector{GeometryBasics.NgonFace{3, Int}}, target_reduction::Float64)
    # Start with a small grid size and increase until we reach target reduction
    grid_size = 0.01
    max_grid_size = 1.0
    target_vertex_count = Int(round(length(vertices) * (1 - target_reduction)))
    
    best_vertices = vertices
    best_faces = faces
    
    while grid_size <= max_grid_size
        clustered_vertices, clustered_faces = vertex_clustering_simplification(vertices, faces, grid_size)
        
        if length(clustered_vertices) <= target_vertex_count
            best_vertices = clustered_vertices
            best_faces = clustered_faces
            break
        end
        
        # If we're still above target, keep this result and continue
        if length(clustered_vertices) < length(best_vertices)
            best_vertices = clustered_vertices
            best_faces = clustered_faces
        end
        
        grid_size *= 1.5  # Increase grid size
    end
    
    return best_vertices, best_faces
end

# Demo the adaptive clustering
function demo_adaptive_clustering()
    # Create original sphere mesh
    S = GeometryBasics.mesh(GeometryBasics.Sphere(Point3f(0.0, 0.0, 0.0), 1.0))
    vertices = S.position
    faces = [GeometryBasics.NgonFace{3, Int}([Int(f[i].i) + 1 for i in 1:3]...) for f in S.faces]
    
    println("Original mesh: $(length(vertices)) vertices, $(length(faces)) faces")
    
    # Test different reduction targets
    reductions = [0.25, 0.5, 0.75, 0.9]
    
    fig = Figure(size = (1200, 800))
    
    # Original mesh
    ax1 = Axis3(fig[1, 1], title="Original Mesh\n$(length(vertices)) vertices")
    mesh!(ax1, S, color=:blue, alpha=0.8)
    
    for (i, reduction) in enumerate(reductions)
        adaptive_vertices, adaptive_faces = adaptive_vertex_clustering(vertices, faces, reduction)
        
        # Create new mesh
        adaptive_mesh = GeometryBasics.Mesh(adaptive_vertices, adaptive_faces)
        
        # Calculate actual reduction percentage
        actual_reduction = round((1 - length(adaptive_vertices) / length(vertices)) * 100, digits=1)
        
        println("Target $(reduction*100)% reduction: $(length(adaptive_vertices)) vertices, $(length(adaptive_faces)) faces (actual: $(actual_reduction)% reduction)")
        
        # Plot
        row = (i - 1) ÷ 2 + 1
        col = (i - 1) % 2 + 2
        ax = Axis3(fig[row, col], title="Target: $(reduction*100)% reduction\n$(length(adaptive_vertices)) vertices (actual: $(actual_reduction)%)")
        mesh!(ax, adaptive_mesh, color=:green, alpha=0.8)
    end
    
    display(fig)
    
    return adaptive_vertices, adaptive_faces
end

# Run the demos
simplified_mesh = demo_vertex_clustering()

Original mesh: 576 vertices, 1058 faces
Grid size 0.1: 468 vertices, 932 faces (18.8% reduction)
Grid size 0.2: 291 vertices, 578 faces (49.5% reduction)
Grid size 0.3: 157 vertices, 310 faces (72.7% reduction)
Grid size 0.5: 57 vertices, 110 faces (90.1% reduction)


Mesh{3, Float32, GeometryBasics.TriangleFace{Int64}}
    faces: 310
    vertex position: 157


In [4]:
# Add wireframe visualization to the vertex clustering demo
function demo_vertex_clustering_with_wireframe()
    # Create original sphere mesh
    S = GeometryBasics.mesh(GeometryBasics.Sphere(Point3f(0.0, 0.0, 0.0), 1.0))
    vertices = S.position
    faces = [GeometryBasics.NgonFace{3, Int}([Int(f[i].i) + 1 for i in 1:3]...) for f in S.faces]
    
    println("Original mesh: $(length(vertices)) vertices, $(length(faces)) faces")
    
    # Apply vertex clustering with different grid sizes
    grid_sizes = [0.1, 0.2, 0.3, 0.5]
    
    fig = Figure(size = (1200, 800))
    
    # Original mesh
    ax1 = Axis3(fig[1, 1], title="Original Mesh\n$(length(vertices)) vertices")
    mesh!(ax1, S, color=:blue, alpha=0.8)
    
    for (i, grid_size) in enumerate(grid_sizes)
        clustered_vertices, clustered_faces = vertex_clustering_simplification(vertices, faces, grid_size)
        
        # Create new mesh
        clustered_mesh = GeometryBasics.Mesh(clustered_vertices, clustered_faces)
        
        # Calculate reduction percentage
        reduction = round((1 - length(clustered_vertices) / length(vertices)) * 100, digits=1)
        
        println("Grid size $grid_size: $(length(clustered_vertices)) vertices, $(length(clustered_faces)) faces ($(reduction)% reduction)")
        
        # Plot
        row = (i - 1) ÷ 2 + 1
        col = (i - 1) % 2 + 2
        ax = Axis3(fig[row, col], title="Grid Size: $grid_size\n$(length(clustered_vertices)) vertices ($(reduction)% reduction)")
        
        # Draw filled mesh
        mesh!(ax, clustered_mesh, color=:red, alpha=0.6)
        
        # Draw wireframe for each face
        for face in clustered_faces
            v1 = clustered_vertices[face[1]]
            v2 = clustered_vertices[face[2]]
            v3 = clustered_vertices[face[3]]
            
            # Draw triangle edges
            lines!(ax, [v1, v2, v3, v1], color=:black, linewidth=1.5)
        end
    end
    
    display(fig)
    
    # Return the most simplified mesh for further use
    final_vertices, final_faces = vertex_clustering_simplification(vertices, faces, 0.3)
    return GeometryBasics.Mesh(final_vertices, final_faces)
end

# Enhanced adaptive clustering with wireframe
function demo_adaptive_clustering_with_wireframe()
    # Create original sphere mesh
    S = GeometryBasics.mesh(GeometryBasics.Sphere(Point3f(0.0, 0.0, 0.0), 1.0))
    vertices = S.position
    faces = [GeometryBasics.NgonFace{3, Int}([Int(f[i].i) + 1 for i in 1:3]...) for f in S.faces]
    
    println("Original mesh: $(length(vertices)) vertices, $(length(faces)) faces")
    
    # Test different reduction targets
    reductions = [0.25, 0.5, 0.75, 0.9]
    
    fig = Figure(size = (1200, 800))
    
    # Original mesh with wireframe
    ax1 = Axis3(fig[1, 1], title="Original Mesh\n$(length(vertices)) vertices")
    mesh!(ax1, S, color=:blue, alpha=0.6)
    
    # Draw original wireframe (sample only first 100 faces for performance)
    for face in faces[1:min(100, length(faces))]
        v1 = vertices[face[1]]
        v2 = vertices[face[2]]
        v3 = vertices[face[3]]
        lines!(ax1, [v1, v2, v3, v1], color=:darkblue, linewidth=0.5)
    end
    
    # Store the final result for return
    final_result = nothing
    
    for (i, reduction) in enumerate(reductions)
        curr_vertices, curr_faces = adaptive_vertex_clustering(vertices, faces, reduction)
        
        # Store the last result
        if i == length(reductions)
            final_result = (curr_vertices, curr_faces)
        end
        
        # Create new mesh
        adaptive_mesh = GeometryBasics.Mesh(curr_vertices, curr_faces)
        
        # Calculate actual reduction percentage
        actual_reduction = round((1 - length(curr_vertices) / length(vertices)) * 100, digits=1)
        
        println("Target $(reduction*100)% reduction: $(length(curr_vertices)) vertices, $(length(curr_faces)) faces (actual: $(actual_reduction)% reduction)")
        
        # Plot
        row = (i - 1) ÷ 2 + 1
        col = (i - 1) % 2 + 2
        ax = Axis3(fig[row, col], title="Target: $(reduction*100)% reduction\n$(length(curr_vertices)) vertices (actual: $(actual_reduction)%)")
        
        # Draw filled mesh
        mesh!(ax, adaptive_mesh, color=:green, alpha=0.6)
        
        # Draw wireframe for each face
        for face in curr_faces
            v1 = curr_vertices[face[1]]
            v2 = curr_vertices[face[2]]
            v3 = curr_vertices[face[3]]
            
            # Draw triangle edges
            lines!(ax, [v1, v2, v3, v1], color=:darkgreen, linewidth=1.5)
        end
    end
    
    display(fig)
    
    return final_result
end

# Function to create wireframe-only visualization
function visualize_wireframe_only(vertices::Vector{Point3f}, faces::Vector{GeometryBasics.NgonFace{3, Int}}, title::String="Wireframe Mesh")
    fig = Figure(size = (800, 600))
    ax = Axis3(fig[1, 1], title=title)
    
    # Draw wireframe for each face
    for face in faces
        v1 = vertices[face[1]]
        v2 = vertices[face[2]]
        v3 = vertices[face[3]]
        
        # Draw triangle edges
        lines!(ax, [v1, v2, v3, v1], color=:black, linewidth=1.0)
    end
    
    display(fig)
    return fig
end
nothing

In [52]:
simplified_mesh = demo_vertex_clustering_with_wireframe()

Original mesh: 576 vertices, 1058 faces
Grid size 0.1: 468 vertices, 932 faces (18.8% reduction)
Grid size 0.2: 291 vertices, 578 faces (49.5% reduction)
Grid size 0.3: 157 vertices, 310 faces (72.7% reduction)
Grid size 0.5: 57 vertices, 110 faces (90.1% reduction)


Mesh{3, Float32, GeometryBasics.TriangleFace{Int64}}
    faces: 310
    vertex position: 157


In [53]:
adaptive_result = demo_adaptive_clustering_with_wireframe()

Original mesh: 576 vertices, 1058 faces
Target 25.0% reduction: 342 vertices, 680 faces (actual: 40.6% reduction)
Target 50.0% reduction: 201 vertices, 398 faces (actual: 65.1% reduction)
Target 75.0% reduction: 106 vertices, 208 faces (actual: 81.6% reduction)
Target 90.0% reduction: 48 vertices, 92 faces (actual: 91.7% reduction)


(GeometryBasics.Point{3, Float32}[[-0.14615677, -0.83344483, 0.4725292], [-0.6803071, -0.11419268, -0.6764999], [-0.12260679, 0.37824452, 0.88836735], [-0.5646684, 0.7999529, -0.1345839], [-0.8419488, -0.1157232, 0.45152676], [0.75862634, -0.46133092, -0.46006504], [0.800087, -0.48654366, 0.33280024], [-0.5894169, 0.44617537, -0.6303114], [-0.17181617, 0.8268248, -0.51837265], [-0.6754416, 0.49810824, 0.479582]  …  [0.43934545, -0.84789824, -0.1345839], [-0.554001, 0.7848405, 0.2691678], [0.43219122, -0.09906625, -0.86144555], [0.32485017, 0.91404074, -0.1345839], [0.6974617, -0.14422695, 0.6825532], [-0.66211104, -0.7089479, -0.1345839], [0.43611488, 0.41239953, -0.75268286], [-0.050210834, -0.054391682, 0.97907054], [0.571221, 0.5559945, 0.5458654], [-0.11510051, -0.64930075, -0.70944494]], GeometryBasics.TriangleFace{Int64}[TriangleFace{Int64}(28, 43, 47), TriangleFace{Int64}(28, 47, 31), TriangleFace{Int64}(43, 22, 25), TriangleFace{Int64}(43, 25, 47), TriangleFace{Int64}(22, 26, 2

In [54]:
wireframe_fig = visualize_wireframe_only(adaptive_result[1], adaptive_result[2], "Wireframe Only - Adaptive Clustering")

In [58]:
mymesh = GeometryBasics.mesh(GeometryBasics.Rect(Point2f(0, 0), Point2f(1, 1)))
mesh(mymesh)

---

In [2]:
using GLMakie
import GLMakie.GeometryBasics


# Vertex clustering mesh simplification
function vertex_clustering_simplification(vertices::Vector{Point3f}, faces::Vector{GeometryBasics.NgonFace{3, Int}}, grid_size::Float64)
    # Create a spatial grid for clustering
    # Find bounding box
    min_coords = Point3f(Inf, Inf, Inf)
    max_coords = Point3f(-Inf, -Inf, -Inf)
    
    for v in vertices
        min_coords = Point3f(min(min_coords[1], v[1]), min(min_coords[2], v[2]), min(min_coords[3], v[3]))
        max_coords = Point3f(max(max_coords[1], v[1]), max(max_coords[2], v[2]), max(max_coords[3], v[3]))
    end
    
    # Create grid cells
    grid_cells = Dict{Tuple{Int, Int, Int}, Vector{Int}}()
    
    # Assign vertices to grid cells
    for (i, v) in enumerate(vertices)
        # Calculate grid cell coordinates
        cell_x = Int(floor((v[1] - min_coords[1]) / grid_size))
        cell_y = Int(floor((v[2] - min_coords[2]) / grid_size))
        cell_z = Int(floor((v[3] - min_coords[3]) / grid_size))
        
        cell_key = (cell_x, cell_y, cell_z)
        
        if !haskey(grid_cells, cell_key)
            grid_cells[cell_key] = Int[]
        end
        push!(grid_cells[cell_key], i)
    end
    
    # Create clustered vertices and mapping
    clustered_vertices = Point3f[]
    vertex_mapping = Dict{Int, Int}()
    
    for (cell_key, vertex_indices) in grid_cells
        if !isempty(vertex_indices)
            # Compute centroid of vertices in this cell
            centroid = Point3f(0.0, 0.0, 0.0)
            for idx in vertex_indices
                centroid = centroid + vertices[idx]
            end
            centroid = centroid / length(vertex_indices)
            
            # Add clustered vertex
            push!(clustered_vertices, centroid)
            cluster_idx = length(clustered_vertices)
            
            # Map all original vertices in this cell to the clustered vertex
            for idx in vertex_indices
                vertex_mapping[idx] = cluster_idx
            end
        end
    end
    
    # Update faces with new vertex indices
    new_faces = GeometryBasics.NgonFace{3, Int}[]
    for face in faces
        new_face_indices = [vertex_mapping[face[i]] for i in 1:3]
        
        # Only keep non-degenerate faces (all vertices different)
        if length(unique(new_face_indices)) == 3
            push!(new_faces, GeometryBasics.NgonFace{3, Int}(new_face_indices...))
        end
    end
    
    return clustered_vertices, new_faces
end

# Advanced vertex clustering with quality preservation
function adaptive_vertex_clustering(vertices::Vector{Point3f}, faces::Vector{GeometryBasics.NgonFace{3, Int}}, target_reduction::Float64)
    # Start with a small grid size and increase until we reach target reduction
    grid_size = 0.01
    max_grid_size = 1.0
    target_vertex_count = Int(round(length(vertices) * (1 - target_reduction)))
    
    best_vertices = vertices
    best_faces = faces
    
    while grid_size <= max_grid_size
        clustered_vertices, clustered_faces = vertex_clustering_simplification(vertices, faces, grid_size)
        
        if length(clustered_vertices) <= target_vertex_count
            best_vertices = clustered_vertices
            best_faces = clustered_faces
            break
        end
        
        # If we're still above target, keep this result and continue
        if length(clustered_vertices) < length(best_vertices)
            best_vertices = clustered_vertices
            best_faces = clustered_faces
        end
        
        grid_size *= 1.5  # Increase grid size
    end
    
    return best_vertices, best_faces
end

"""
    simplify(mesh::GeometryBasics.Mesh, intensity::Float64, algorithm::Symbol)

Simplify a mesh using the specified algorithm and intensity.

# Arguments
- `mesh`: The input GeometryBasics.Mesh to simplify
- `intensity`: Simplification intensity factor
  - For `:simple`: grid size (0.01 to 1.0, higher = more aggressive)
  - For `:adaptive`: reduction percentage (0.0 to 1.0, where 0.5 = 50% reduction)
- `algorithm`: Either `:simple` or `:adaptive`

# Returns
- `GeometryBasics.Mesh`: The simplified mesh

# Examples
```julia
# Simple vertex clustering with grid size 0.3
simplified = simplify(mesh, 0.3, :simple)

# Adaptive clustering with 75% vertex reduction
simplified = simplify(mesh, 0.75, :adaptive)
```
"""
function simplify(mesh::GeometryBasics.Mesh, intensity::Float64, algorithm::Symbol)
    # Extract vertices and faces from input mesh
    vertices = mesh.position
    faces = [GeometryBasics.NgonFace{3, Int}([Int(f[i].i) + 1 for i in 1:3]...) for f in mesh.faces]
    
    # Validate algorithm parameter
    if algorithm ∉ [:simple, :adaptive]
        throw(ArgumentError("Algorithm must be :simple or :adaptive, got :$algorithm"))
    end
    
    # Validate intensity parameter
    if intensity < 0.0 || intensity > 1.0
        throw(ArgumentError("Intensity must be between 0.0 and 1.0, got $intensity"))
    end
    
    # Apply the appropriate simplification algorithm
    if algorithm == :simple
        # For simple algorithm, intensity is the grid size
        # Map intensity (0.0-1.0) to reasonable grid size range (0.01-1.0)
        grid_size = 0.01 + intensity * 0.99
        
        println("Simplifying mesh using vertex clustering with grid size: $grid_size")
        simplified_vertices, simplified_faces = vertex_clustering_simplification(vertices, faces, grid_size)
        
    elseif algorithm == :adaptive
        # For adaptive algorithm, intensity is the reduction percentage
        target_reduction = intensity
        
        println("Simplifying mesh using adaptive clustering with $(target_reduction*100)% reduction target")
        simplified_vertices, simplified_faces = adaptive_vertex_clustering(vertices, faces, target_reduction)
    end
    
    # Create and return the simplified mesh
    simplified_mesh = GeometryBasics.Mesh(simplified_vertices, simplified_faces)
    
    # Print simplification results
    original_vertex_count = length(vertices)
    simplified_vertex_count = length(simplified_vertices)
    actual_reduction = round((1 - simplified_vertex_count / original_vertex_count) * 100, digits=1)
    
    println("Simplification complete:")
    println("  Original vertices: $original_vertex_count")
    println("  Simplified vertices: $simplified_vertex_count")
    println("  Actual reduction: $(actual_reduction)%")
    
    return simplified_mesh
end

# Convenience function for quick testing
function demo_simplify_function()
    # Create a test mesh
    S = GeometryBasics.mesh(GeometryBasics.Sphere(Point3f(0.0, 0.0, 0.0), 1.0))
    
    println("=== Testing simplify function ===")
    println("Original mesh: $(length(S.position)) vertices, $(length(S.faces)) faces")
    
    # Test simple algorithm with different intensities
    println("\n--- Simple Algorithm Tests ---")
    simple_low = simplify(S, 0.2, :simple)      # Low intensity (small grid)
    simple_medium = simplify(S, 0.5, :simple)   # Medium intensity
    simple_high = simplify(S, 0.8, :simple)     # High intensity (large grid)
    
    # Test adaptive algorithm with different intensities
    println("\n--- Adaptive Algorithm Tests ---")
    adaptive_25 = simplify(S, 0.25, :adaptive)   # 25% reduction
    adaptive_50 = simplify(S, 0.50, :adaptive)   # 50% reduction
    adaptive_75 = simplify(S, 0.75, :adaptive)   # 75% reduction
    
    # Visualize results
    fig = Figure(size = (1500, 1000))
    
    # Original
    ax1 = Axis3(fig[1, 1], title="Original\n$(length(S.position)) vertices")
    mesh!(ax1, S, color=:blue, alpha=0.8)
    
    # Simple results
    ax2 = Axis3(fig[1, 2], title="Simple (0.2)\n$(length(simple_low.position)) vertices")
    mesh!(ax2, simple_low, color=:red, alpha=0.8)
    
    ax3 = Axis3(fig[1, 3], title="Simple (0.5)\n$(length(simple_medium.position)) vertices")
    mesh!(ax3, simple_medium, color=:red, alpha=0.8)
    
    ax4 = Axis3(fig[1, 4], title="Simple (0.8)\n$(length(simple_high.position)) vertices")
    mesh!(ax4, simple_high, color=:red, alpha=0.8)
    
    # Adaptive results
    ax5 = Axis3(fig[2, 1], title="Adaptive (25%)\n$(length(adaptive_25.position)) vertices")
    mesh!(ax5, adaptive_25, color=:green, alpha=0.8)
    
    ax6 = Axis3(fig[2, 2], title="Adaptive (50%)\n$(length(adaptive_50.position)) vertices")
    mesh!(ax6, adaptive_50, color=:green, alpha=0.8)
    
    ax7 = Axis3(fig[2, 3], title="Adaptive (75%)\n$(length(adaptive_75.position)) vertices")
    mesh!(ax7, adaptive_75, color=:green, alpha=0.8)
    
    display(fig)
    
    return (simple_low, simple_medium, simple_high, adaptive_25, adaptive_50, adaptive_75)
end

# Run the demo
# test_results = demo_simplify_function()

demo_simplify_function (generic function with 1 method)

In [3]:
# Create a mesh
S = GeometryBasics.mesh(GeometryBasics.Sphere(Point3f(0.0, 0.0, 0.0), 1.0))
println()
# Simple vertex clustering with low intensity (fine grid)
simplified_fine = simplify(S, 0.2, :simple)
println()

# Simple vertex clustering with high intensity (coarse grid)
simplified_coarse = simplify(S, 0.8, :simple)
println()

# Adaptive clustering for 50% vertex reduction
simplified_half = simplify(S, 0.5, :adaptive)
println()

# Adaptive clustering for aggressive 90% reduction
simplified_aggressive = simplify(S, 0.9, :adaptive)


Simplifying mesh using vertex clustering with grid size: 0.20800000000000002
Simplification complete:
  Original vertices: 576
  Simplified vertices: 277
  Actual reduction: 51.9%

Simplifying mesh using vertex clustering with grid size: 0.802
Simplification complete:
  Original vertices: 576
  Simplified vertices: 26
  Actual reduction: 95.5%

Simplifying mesh using adaptive clustering with 50.0% reduction target
Simplification complete:
  Original vertices: 576
  Simplified vertices: 201
  Actual reduction: 65.1%

Simplifying mesh using adaptive clustering with 90.0% reduction target
Simplification complete:
  Original vertices: 576
  Simplified vertices: 48
  Actual reduction: 91.7%


Mesh{3, Float32, GeometryBasics.TriangleFace{Int64}}
    faces: 92
    vertex position: 48


In [4]:
function plot_with_wireframe(m)
    f, ax, pl = mesh(m)
    wires = wireframe!(ax, m, color=:black)
    display(f)
    return f, ax, pl, wires
end

plot_with_wireframe (generic function with 1 method)

In [5]:
plot_with_wireframe(S)

(Scene (600px, 450px):
  0 Plots
  1 Child Scene:
    └ Scene (600px, 450px), LScene(), Mesh{Tuple{GeometryBasics.Mesh{3, Float32, GeometryBasics.NgonFace{3, GeometryBasics.OffsetInteger{-1, UInt32}}, (:position, :normal), Tuple{Vector{Point{3, Float32}}, Vector{Vec{3, Float32}}}, Vector{GeometryBasics.NgonFace{3, GeometryBasics.OffsetInteger{-1, UInt32}}}}}}, Wireframe{Tuple{GeometryBasics.Mesh{3, Float32, GeometryBasics.NgonFace{3, GeometryBasics.OffsetInteger{-1, UInt32}}, (:position,), Tuple{Vector{Point{3, Float32}}}, Vector{GeometryBasics.NgonFace{3, GeometryBasics.OffsetInteger{-1, UInt32}}}}}})

In [6]:
plot_with_wireframe(simplified_fine)

(Scene (600px, 450px):
  0 Plots
  1 Child Scene:
    └ Scene (600px, 450px), LScene(), Mesh{Tuple{GeometryBasics.Mesh{3, Float32, GeometryBasics.NgonFace{3, GeometryBasics.OffsetInteger{-1, UInt32}}, (:position, :normal), Tuple{Vector{Point{3, Float32}}, Vector{Vec{3, Float32}}}, Vector{GeometryBasics.NgonFace{3, GeometryBasics.OffsetInteger{-1, UInt32}}}}}}, Wireframe{Tuple{GeometryBasics.Mesh{3, Float32, GeometryBasics.TriangleFace{Int64}, (:position,), Tuple{Vector{Point{3, Float32}}}, Vector{GeometryBasics.TriangleFace{Int64}}}}})

In [12]:
plot_with_wireframe(simplified_coarse)

(Scene(1 children, 0 plots), LScene(), Mesh{Tuple{GeometryBasics.Mesh{3, Float32, GeometryBasics.NgonFace{3, GeometryBasics.OffsetInteger{-1, UInt32}}, (:position, :normal), Tuple{Vector{Point{3, Float32}}, Vector{Vec{3, Float32}}}, Vector{GeometryBasics.NgonFace{3, GeometryBasics.OffsetInteger{-1, UInt32}}}}}}, Wireframe{Tuple{GeometryBasics.Mesh{3, Float32, GeometryBasics.TriangleFace{Int64}, (:position,), Tuple{Vector{Point{3, Float32}}}, Vector{GeometryBasics.TriangleFace{Int64}}}}})

In [13]:
plot_with_wireframe(simplified_half)

(Scene(1 children, 0 plots), LScene(), Mesh{Tuple{GeometryBasics.Mesh{3, Float32, GeometryBasics.NgonFace{3, GeometryBasics.OffsetInteger{-1, UInt32}}, (:position, :normal), Tuple{Vector{Point{3, Float32}}, Vector{Vec{3, Float32}}}, Vector{GeometryBasics.NgonFace{3, GeometryBasics.OffsetInteger{-1, UInt32}}}}}}, Wireframe{Tuple{GeometryBasics.Mesh{3, Float32, GeometryBasics.TriangleFace{Int64}, (:position,), Tuple{Vector{Point{3, Float32}}}, Vector{GeometryBasics.TriangleFace{Int64}}}}})

In [14]:
plot_with_wireframe(simplified_aggressive)

(Scene(1 children, 0 plots), LScene(), Mesh{Tuple{GeometryBasics.Mesh{3, Float32, GeometryBasics.NgonFace{3, GeometryBasics.OffsetInteger{-1, UInt32}}, (:position, :normal), Tuple{Vector{Point{3, Float32}}, Vector{Vec{3, Float32}}}, Vector{GeometryBasics.NgonFace{3, GeometryBasics.OffsetInteger{-1, UInt32}}}}}}, Wireframe{Tuple{GeometryBasics.Mesh{3, Float32, GeometryBasics.TriangleFace{Int64}, (:position,), Tuple{Vector{Point{3, Float32}}}, Vector{GeometryBasics.TriangleFace{Int64}}}}})

In [15]:
plot_with_wireframe(simplify(S, 0.9, :adaptive))

Simplifying mesh using adaptive clustering with 90.0% reduction target
Simplification complete:
  Original vertices: 576
  Simplified vertices: 48
  Actual reduction: 91.7%


(Scene(1 children, 0 plots), LScene(), Mesh{Tuple{GeometryBasics.Mesh{3, Float32, GeometryBasics.NgonFace{3, GeometryBasics.OffsetInteger{-1, UInt32}}, (:position, :normal), Tuple{Vector{Point{3, Float32}}, Vector{Vec{3, Float32}}}, Vector{GeometryBasics.NgonFace{3, GeometryBasics.OffsetInteger{-1, UInt32}}}}}}, Wireframe{Tuple{GeometryBasics.Mesh{3, Float32, GeometryBasics.TriangleFace{Int64}, (:position,), Tuple{Vector{Point{3, Float32}}}, Vector{GeometryBasics.TriangleFace{Int64}}}}})

In [7]:
using FileIO

# _wing_model = load("../Models/DragonFly/FrontRight.stl") # Load the shader for the body
_wing_model = load("../Models/DragonFly/HighQuality/frontrightwing.obj") # Load the shader for the wing
_wing_model = GeometryBasics.mesh(_wing_model)

Mesh{3, Float32, GeometryBasics.NgonFace{3, GeometryBasics.OffsetInteger{-1, UInt32}}}
    faces: 18240
    vertex position: 9122


In [8]:
simpled_wing = simplify(_wing_model, 0.99, :adaptive)

Simplifying mesh using adaptive clustering with 99.0% reduction target
Simplification complete:
  Original vertices: 9122
  Simplified vertices: 64
  Actual reduction: 99.3%


Mesh{3, Float32, GeometryBasics.TriangleFace{Int64}}
    faces: 166
    vertex position: 64


In [9]:
_wing_model

Mesh{3, Float32, GeometryBasics.NgonFace{3, GeometryBasics.OffsetInteger{-1, UInt32}}}
    faces: 18240
    vertex position: 9122


In [10]:
simpled_wing

Mesh{3, Float32, GeometryBasics.TriangleFace{Int64}}
    faces: 166
    vertex position: 64


In [11]:
plot_with_wireframe(simpled_wing)

(Scene (600px, 450px):
  0 Plots
  1 Child Scene:
    └ Scene (600px, 450px), LScene(), Mesh{Tuple{GeometryBasics.Mesh{3, Float32, GeometryBasics.NgonFace{3, GeometryBasics.OffsetInteger{-1, UInt32}}, (:position, :normal), Tuple{Vector{Point{3, Float32}}, Vector{Vec{3, Float32}}}, Vector{GeometryBasics.NgonFace{3, GeometryBasics.OffsetInteger{-1, UInt32}}}}}}, Wireframe{Tuple{GeometryBasics.Mesh{3, Float32, GeometryBasics.TriangleFace{Int64}, (:position,), Tuple{Vector{Point{3, Float32}}}, Vector{GeometryBasics.TriangleFace{Int64}}}}})

In [8]:
plot_with_wireframe(simplify(_wing_model, 0.2, :simple))

Simplifying mesh using vertex clustering with grid size: 0.20800000000000002
Simplification complete:
  Original vertices: 9122
  Simplified vertices: 88
  Actual reduction: 99.0%


(Scene (600px, 450px):
  0 Plots
  1 Child Scene:
    └ Scene (600px, 450px), LScene(), Mesh{Tuple{GeometryBasics.Mesh{3, Float32, GeometryBasics.NgonFace{3, GeometryBasics.OffsetInteger{-1, UInt32}}, (:position, :normal), Tuple{Vector{Point{3, Float32}}, Vector{Vec{3, Float32}}}, Vector{GeometryBasics.NgonFace{3, GeometryBasics.OffsetInteger{-1, UInt32}}}}}}, Wireframe{Tuple{GeometryBasics.Mesh{3, Float32, GeometryBasics.TriangleFace{Int64}, (:position,), Tuple{Vector{Point{3, Float32}}}, Vector{GeometryBasics.TriangleFace{Int64}}}}})

In [40]:
plot_at = plot_with_wireframe(simplify(_wing_model, 0.15, :simple))

Simplifying mesh using vertex clustering with grid size: 0.1585
Simplification complete:
  Original vertices: 9122
  Simplified vertices: 154
  Actual reduction: 98.3%


(Scene (600px, 450px):
  0 Plots
  1 Child Scene:
    └ Scene (600px, 450px), LScene(), Mesh{Tuple{GeometryBasics.Mesh{3, Float32, GeometryBasics.NgonFace{3, GeometryBasics.OffsetInteger{-1, UInt32}}, (:position, :normal), Tuple{Vector{Point{3, Float32}}, Vector{Vec{3, Float32}}}, Vector{GeometryBasics.NgonFace{3, GeometryBasics.OffsetInteger{-1, UInt32}}}}}}, Wireframe{Tuple{GeometryBasics.Mesh{3, Float32, GeometryBasics.TriangleFace{Int64}, (:position,), Tuple{Vector{Point{3, Float32}}}, Vector{GeometryBasics.TriangleFace{Int64}}}}})

In [41]:
mesh!(plot_at[2], _wing_model, color=:red, alpha=0.5)

Mesh{Tuple{GeometryBasics.Mesh{3, Float32, GeometryBasics.NgonFace{3, GeometryBasics.OffsetInteger{-1, UInt32}}, (:position, :normal), Tuple{Vector{Point{3, Float32}}, Vector{Vec{3, Float32}}}, Vector{GeometryBasics.NgonFace{3, GeometryBasics.OffsetInteger{-1, UInt32}}}}}}

---

In [16]:

using LinearAlgebra
using Statistics: mean, std

# Calculate triangle angles in degrees
function triangle_angles(v1::Point3f, v2::Point3f, v3::Point3f)
    # Edge vectors
    a = v2 - v1
    b = v3 - v1
    c = v3 - v2
    
    # Edge lengths
    len_a = norm(a)
    len_b = norm(b)
    len_c = norm(c)
    
    # Prevent division by zero
    if len_a < 1e-10 || len_b < 1e-10 || len_c < 1e-10
        return [0.0, 0.0, 0.0]
    end
    
    # Calculate angles using law of cosines
    angle1 = acos(clamp(dot(a, b) / (len_a * len_b), -1.0, 1.0))
    angle2 = acos(clamp(dot(-a, c) / (len_a * len_c), -1.0, 1.0))
    angle3 = π - angle1 - angle2
    
    # Convert to degrees
    return [rad2deg(angle1), rad2deg(angle2), rad2deg(angle3)]
end

# Calculate triangle quality based on angle deviation from 60 degrees
function triangle_angle_quality(v1::Point3f, v2::Point3f, v3::Point3f)
    angles = triangle_angles(v1, v2, v3)
    
    # Calculate deviation from 60 degrees
    deviations = abs.(angles .- 60.0)
    
    # Quality score: lower is better (0 = perfect equilateral triangle)
    # Maximum possible deviation is 120 degrees (degenerate triangle)
    quality = sum(deviations) / 180.0  # Normalize to [0, 1]
    
    return quality
end

# Quality-based vertex clustering with angle optimization
function angle_optimized_clustering(vertices::Vector{Point3f}, faces::Vector{GeometryBasics.NgonFace{3, Int}}, target_reduction::Float64)
    # Start with a copy of the original mesh
    current_vertices = copy(vertices)
    current_faces = copy(faces)
    target_vertex_count = Int(round(length(vertices) * (1 - target_reduction)))
    
    # Track which vertices can be merged
    vertex_quality = Dict{Int, Float64}()
    
    # Calculate initial quality for all vertices
    for (i, vertex) in enumerate(current_vertices)
        # Find faces containing this vertex
        adjacent_faces = [f for f in current_faces if i in f]
        
        if !isempty(adjacent_faces)
            # Calculate average quality of adjacent triangles
            total_quality = 0.0
            for face in adjacent_faces
                v1 = current_vertices[face[1]]
                v2 = current_vertices[face[2]]
                v3 = current_vertices[face[3]]
                total_quality += triangle_angle_quality(v1, v2, v3)
            end
            vertex_quality[i] = total_quality / length(adjacent_faces)
        else
            vertex_quality[i] = 1.0  # Worst quality for isolated vertices
        end
    end
    
    # Iteratively remove vertices with worst quality impact
    while length(current_vertices) > target_vertex_count
        # Find vertex pairs that can be merged
        merge_candidates = []
        
        for i in 1:length(current_vertices)
            for j in (i+1):length(current_vertices)
                if i != j
                    # Calculate distance between vertices
                    dist = norm(current_vertices[i] - current_vertices[j])
                    
                    # Only consider nearby vertices for merging
                    if dist < 0.1  # Adjustable threshold
                        # Calculate quality improvement if we merge these vertices
                        new_vertex = (current_vertices[i] + current_vertices[j]) / 2
                        
                        # Find faces that would be affected
                        affected_faces = [f for f in current_faces if i in f || j in f]
                        
                        # Calculate current quality
                        current_quality = 0.0
                        valid_faces = 0
                        for face in affected_faces
                            v1 = current_vertices[face[1]]
                            v2 = current_vertices[face[2]]
                            v3 = current_vertices[face[3]]
                            current_quality += triangle_angle_quality(v1, v2, v3)
                            valid_faces += 1
                        end
                        
                        if valid_faces > 0
                            current_quality /= valid_faces
                            
                            # Calculate new quality after merge
                            new_quality = 0.0
                            new_valid_faces = 0
                            
                            for face in affected_faces
                                # Create new face with merged vertex
                                new_face_vertices = []
                                for vertex_idx in face
                                    if vertex_idx == i || vertex_idx == j
                                        push!(new_face_vertices, new_vertex)
                                    else
                                        push!(new_face_vertices, current_vertices[vertex_idx])
                                    end
                                end
                                
                                # Skip degenerate triangles
                                if length(unique(new_face_vertices)) == 3
                                    new_quality += triangle_angle_quality(new_face_vertices[1], new_face_vertices[2], new_face_vertices[3])
                                    new_valid_faces += 1
                                end
                            end
                            
                            if new_valid_faces > 0
                                new_quality /= new_valid_faces
                                quality_improvement = current_quality - new_quality
                                
                                push!(merge_candidates, (i, j, quality_improvement, new_vertex))
                            end
                        end
                    end
                end
            end
        end
        
        # If no good merge candidates, break
        if isempty(merge_candidates)
            break
        end
        
        # Sort by quality improvement (highest first)
        sort!(merge_candidates, by=x->x[3], rev=true)
        
        # Perform the best merge
        best_merge = merge_candidates[1]
        i, j, improvement, new_vertex = best_merge
        
        # Update vertices
        new_vertices = Point3f[]
        vertex_mapping = Dict{Int, Int}()
        
        for (idx, vertex) in enumerate(current_vertices)
            if idx == i
                push!(new_vertices, new_vertex)
                vertex_mapping[idx] = length(new_vertices)
            elseif idx == j
                vertex_mapping[idx] = vertex_mapping[i]
            else
                push!(new_vertices, vertex)
                vertex_mapping[idx] = length(new_vertices)
            end
        end
        
        # Update faces
        new_faces = GeometryBasics.NgonFace{3, Int}[]
        for face in current_faces
            new_face_indices = [vertex_mapping[face[k]] for k in 1:3]
            
            # Only keep non-degenerate faces
            if length(unique(new_face_indices)) == 3
                push!(new_faces, GeometryBasics.NgonFace{3, Int}(new_face_indices...))
            end
        end
        
        current_vertices = new_vertices
        current_faces = new_faces
    end
    
    return current_vertices, current_faces
end

# Enhanced simplify function with angle optimization
function simplify(mesh::GeometryBasics.Mesh, intensity::Float64, algorithm::Symbol)
    # Extract vertices and faces from input mesh
    vertices = mesh.position
    faces = mesh.faces
        # Handle different face types
    if !isempty(faces)
        # Check if faces need conversion from OffsetInteger to Int
        first_face = faces[1]
        if hasfield(typeof(first_face[1]), :i)
            # Convert from OffsetInteger to Int (0-based to 1-based)
            faces = [GeometryBasics.NgonFace{3, Int}([Int(f[i].i) + 1 for i in 1:3]...) for f in faces]
        else
            # Already Int type, keep as is
            faces = [GeometryBasics.NgonFace{3, Int}([Int(f[i]) for i in 1:3]...) for f in faces]
        end
    end
    
    # Validate algorithm parameter
    if algorithm ∉ [:simple, :adaptive, :angle_optimized]
        throw(ArgumentError("Algorithm must be :simple, :adaptive, or :angle_optimized, got :$algorithm"))
    end
    
    # Validate intensity parameter
    if intensity < 0.0 || intensity > 1.0
        throw(ArgumentError("Intensity must be between 0.0 and 1.0, got $intensity"))
    end
    
    # Apply the appropriate simplification algorithm
    if algorithm == :simple
        # For simple algorithm, intensity is the grid size
        grid_size = 0.01 + intensity * 0.99
        
        println("Simplifying mesh using vertex clustering with grid size: $grid_size")
        simplified_vertices, simplified_faces = vertex_clustering_simplification(vertices, faces, grid_size)
        
    elseif algorithm == :adaptive
        # For adaptive algorithm, intensity is the reduction percentage
        target_reduction = intensity
        
        println("Simplifying mesh using adaptive clustering with $(target_reduction*100)% reduction target")
        simplified_vertices, simplified_faces = adaptive_vertex_clustering(vertices, faces, target_reduction)
        
    elseif algorithm == :angle_optimized
        # For angle-optimized algorithm, intensity is the reduction percentage
        target_reduction = intensity
        
        println("Simplifying mesh using angle-optimized clustering with $(target_reduction*100)% reduction target")
        simplified_vertices, simplified_faces = angle_optimized_clustering(vertices, faces, target_reduction)
    end
    
    # Create and return the simplified mesh
    simplified_mesh = GeometryBasics.Mesh(simplified_vertices, simplified_faces)
    
    # Print simplification results
    original_vertex_count = length(vertices)
    simplified_vertex_count = length(simplified_vertices)
    actual_reduction = round((1 - simplified_vertex_count / original_vertex_count) * 100, digits=1)
    
    println("Simplification complete:")
    println("  Original vertices: $original_vertex_count")
    println("  Simplified vertices: $simplified_vertex_count")
    println("  Actual reduction: $(actual_reduction)%")
    
    # Calculate and report triangle quality
    if !isempty(simplified_faces)
        total_quality = 0.0
        for face in simplified_faces
            v1 = simplified_vertices[face[1]]
            v2 = simplified_vertices[face[2]]
            v3 = simplified_vertices[face[3]]
            total_quality += triangle_angle_quality(v1, v2, v3)
        end
        avg_quality = total_quality / length(simplified_faces)
        println("  Average triangle quality: $(round(avg_quality, digits=3)) (0 = perfect equilateral)")
    end
    
    return simplified_mesh
end

# Function to analyze triangle quality
function analyze_triangle_quality(mesh::GeometryBasics.Mesh)
    vertices = mesh.position
    faces = mesh.faces
    
    # Handle different face types
    if !isempty(faces)
        # Check if faces need conversion from OffsetInteger to Int
        first_face = faces[1]
        if hasfield(typeof(first_face[1]), :i)
            # Convert from OffsetInteger to Int (0-based to 1-based)
            faces = [GeometryBasics.NgonFace{3, Int}([Int(f[i].i) + 1 for i in 1:3]...) for f in faces]
        else
            # Already Int type, keep as is
            faces = [GeometryBasics.NgonFace{3, Int}([Int(f[i]) for i in 1:3]...) for f in faces]
        end
    end
    
    qualities = Float64[]
    angle_deviations = Float64[]
    
    for face in faces
        v1 = vertices[face[1]]
        v2 = vertices[face[2]]
        v3 = vertices[face[3]]
        
        quality = triangle_angle_quality(v1, v2, v3)
        push!(qualities, quality)
        
        angles = triangle_angles(v1, v2, v3)
        deviations = abs.(angles .- 60.0)
        append!(angle_deviations, deviations)
    end
    
    println("Triangle Quality Analysis:")
    println("  Average quality: $(round(mean(qualities), digits=3))")
    println("  Best quality: $(round(minimum(qualities), digits=3))")
    println("  Worst quality: $(round(maximum(qualities), digits=3))")
    println("  Average angle deviation from 60°: $(round(mean(angle_deviations), digits=1))°")
    println("  Max angle deviation: $(round(maximum(angle_deviations), digits=1))°")
    
    return qualities, angle_deviations
end

# Demo function comparing all algorithms
function demo_all_algorithms()
    # Create a test mesh
    S = GeometryBasics.mesh(GeometryBasics.Sphere(Point3f(0.0, 0.0, 0.0), 1.0))
    
    println("=== Comparing All Simplification Algorithms ===")
    println("Original mesh: $(length(S.position)) vertices, $(length(S.faces)) faces")
    
    # Test all algorithms with 50% reduction
    println("\n--- Original Mesh Quality ---")
    analyze_triangle_quality(S)
    
    println("\n--- Simple Algorithm (50% reduction) ---")
    simple_result = simplify(S, 0.5, :simple)
    analyze_triangle_quality(simple_result)
    
    println("\n--- Adaptive Algorithm (50% reduction) ---")
    adaptive_result = simplify(S, 0.5, :adaptive)
    analyze_triangle_quality(adaptive_result)
    
    println("\n--- Angle-Optimized Algorithm (50% reduction) ---")
    angle_result = simplify(S, 0.5, :angle_optimized)
    analyze_triangle_quality(angle_result)
    
    # Visualize results
    fig = Figure(size = (1200, 800))
    
    # Original
    ax1 = Axis3(fig[1, 1], title="Original\n$(length(S.position)) vertices")
    mesh!(ax1, S, color=:blue, alpha=0.8)
    
    # Simple
    ax2 = Axis3(fig[1, 2], title="Simple\n$(length(simple_result.position)) vertices")
    mesh!(ax2, simple_result, color=:red, alpha=0.8)
    
    # Adaptive
    ax3 = Axis3(fig[2, 1], title="Adaptive\n$(length(adaptive_result.position)) vertices")
    mesh!(ax3, adaptive_result, color=:green, alpha=0.8)
    
    # Angle-optimized
    ax4 = Axis3(fig[2, 2], title="Angle-Optimized\n$(length(angle_result.position)) vertices")
    mesh!(ax4, angle_result, color=:purple, alpha=0.8)
    
    display(fig)
    
    return S, simple_result, adaptive_result, angle_result
end

# Run the comprehensive demo
# results = demo_all_algorithms()

demo_all_algorithms (generic function with 1 method)

In [17]:
demo_all_algorithms()

=== Comparing All Simplification Algorithms ===
Original mesh: 576 vertices, 1058 faces

--- Original Mesh Quality ---
Triangle Quality Analysis:
  Average quality: 0.389
  Best quality: 0.251
  Worst quality: 1.0
  Average angle deviation from 60°: 23.3°
  Max angle deviation: 60.0°

--- Simple Algorithm (50% reduction) ---
Simplifying mesh using vertex clustering with grid size: 0.505
Simplification complete:
  Original vertices: 576
  Simplified vertices: 54
  Actual reduction: 90.6%
  Average triangle quality: 0.31 (0 = perfect equilateral)
Triangle Quality Analysis:
  Average quality: 0.31
  Best quality: 0.024
  Worst quality: 0.936
  Average angle deviation from 60°: 18.6°
  Max angle deviation: 84.3°

--- Adaptive Algorithm (50% reduction) ---
Simplifying mesh using adaptive clustering with 50.0% reduction target
Simplification complete:
  Original vertices: 576
  Simplified vertices: 201
  Actual reduction: 65.1%
  Average triangle quality: 0.335 (0 = perfect equilateral)
Tria

(Mesh{3, Float32, GeometryBasics.NgonFace{3, GeometryBasics.OffsetInteger{-1, UInt32}}}(...), Mesh{3, Float32, GeometryBasics.TriangleFace{Int64}}(...), Mesh{3, Float32, GeometryBasics.TriangleFace{Int64}}(...), Mesh{3, Float32, GeometryBasics.TriangleFace{Int64}}(...))

In [51]:
simplified_wing = simplify(_wing_model, 0.7, :adaptive)

Simplifying mesh using adaptive clustering with 70.0% reduction target
Simplification complete:
  Original vertices: 9122
  Simplified vertices: 2168
  Actual reduction: 76.2%
  Average triangle quality: 0.332 (0 = perfect equilateral)


Mesh{3, Float32, GeometryBasics.TriangleFace{Int64}}
    faces: 7888
    vertex position: 2168


In [52]:
angle_simplified_wing = simplify(simplified_wing, 0.1, :angle_optimized)
plot_with_wireframe(angle_simplified_wing)


Simplifying mesh using angle-optimized clustering with 10.0% reduction target
Simplification complete:
  Original vertices: 2168
  Simplified vertices: 1951
  Actual reduction: 10.0%
  Average triangle quality: 0.281 (0 = perfect equilateral)


(Scene (600px, 450px):
  0 Plots
  1 Child Scene:
    └ Scene (600px, 450px), LScene(), Mesh{Tuple{GeometryBasics.Mesh{3, Float32, GeometryBasics.NgonFace{3, GeometryBasics.OffsetInteger{-1, UInt32}}, (:position, :normal), Tuple{Vector{Point{3, Float32}}, Vector{Vec{3, Float32}}}, Vector{GeometryBasics.NgonFace{3, GeometryBasics.OffsetInteger{-1, UInt32}}}}}}, Wireframe{Tuple{GeometryBasics.Mesh{3, Float32, GeometryBasics.TriangleFace{Int64}, (:position,), Tuple{Vector{Point{3, Float32}}}, Vector{GeometryBasics.TriangleFace{Int64}}}}})

In [53]:
simplified_wing = simplify(angle_simplified_wing, 0.1, :adaptive)
angle_simplified_wing = simplify(simplified_wing, 0.1, :angle_optimized)
plot_with_wireframe(angle_simplified_wing)

Simplifying mesh using adaptive clustering with 10.0% reduction target
Simplification complete:
  Original vertices: 1951
  Simplified vertices: 1138
  Actual reduction: 41.7%
  Average triangle quality: 0.28 (0 = perfect equilateral)
Simplifying mesh using angle-optimized clustering with 10.0% reduction target
Simplification complete:
  Original vertices: 1138
  Simplified vertices: 1024
  Actual reduction: 10.0%
  Average triangle quality: 0.223 (0 = perfect equilateral)


(Scene (600px, 450px):
  0 Plots
  1 Child Scene:
    └ Scene (600px, 450px), LScene(), Mesh{Tuple{GeometryBasics.Mesh{3, Float32, GeometryBasics.NgonFace{3, GeometryBasics.OffsetInteger{-1, UInt32}}, (:position, :normal), Tuple{Vector{Point{3, Float32}}, Vector{Vec{3, Float32}}}, Vector{GeometryBasics.NgonFace{3, GeometryBasics.OffsetInteger{-1, UInt32}}}}}}, Wireframe{Tuple{GeometryBasics.Mesh{3, Float32, GeometryBasics.TriangleFace{Int64}, (:position,), Tuple{Vector{Point{3, Float32}}}, Vector{GeometryBasics.TriangleFace{Int64}}}}})

In [54]:
simplified_wing = simplify(angle_simplified_wing, 0.1, :adaptive)
angle_simplified_wing = simplify(simplified_wing, 0.1, :angle_optimized)
plot_with_wireframe(angle_simplified_wing)

Simplifying mesh using adaptive clustering with 10.0% reduction target
Simplification complete:
  Original vertices: 1024
  Simplified vertices: 883
  Actual reduction: 13.8%
  Average triangle quality: 0.243 (0 = perfect equilateral)
Simplifying mesh using angle-optimized clustering with 10.0% reduction target
Simplification complete:
  Original vertices: 883
  Simplified vertices: 795
  Actual reduction: 10.0%
  Average triangle quality: 0.201 (0 = perfect equilateral)


(Scene (600px, 450px):
  0 Plots
  1 Child Scene:
    └ Scene (600px, 450px), LScene(), Mesh{Tuple{GeometryBasics.Mesh{3, Float32, GeometryBasics.NgonFace{3, GeometryBasics.OffsetInteger{-1, UInt32}}, (:position, :normal), Tuple{Vector{Point{3, Float32}}, Vector{Vec{3, Float32}}}, Vector{GeometryBasics.NgonFace{3, GeometryBasics.OffsetInteger{-1, UInt32}}}}}}, Wireframe{Tuple{GeometryBasics.Mesh{3, Float32, GeometryBasics.TriangleFace{Int64}, (:position,), Tuple{Vector{Point{3, Float32}}}, Vector{GeometryBasics.TriangleFace{Int64}}}}})

In [55]:
simplified_wing = simplify(angle_simplified_wing, 0.1, :adaptive)
angle_simplified_wing = simplify(simplified_wing, 0.1, :angle_optimized)
plot_with_wireframe(angle_simplified_wing)

Simplifying mesh using adaptive clustering with 10.0% reduction target
Simplification complete:
  Original vertices: 795
  Simplified vertices: 509
  Actual reduction: 36.0%
  Average triangle quality: 0.271 (0 = perfect equilateral)
Simplifying mesh using angle-optimized clustering with 10.0% reduction target
Simplification complete:
  Original vertices: 509
  Simplified vertices: 458
  Actual reduction: 10.0%
  Average triangle quality: 0.223 (0 = perfect equilateral)


(Scene (600px, 450px):
  0 Plots
  1 Child Scene:
    └ Scene (600px, 450px), LScene(), Mesh{Tuple{GeometryBasics.Mesh{3, Float32, GeometryBasics.NgonFace{3, GeometryBasics.OffsetInteger{-1, UInt32}}, (:position, :normal), Tuple{Vector{Point{3, Float32}}, Vector{Vec{3, Float32}}}, Vector{GeometryBasics.NgonFace{3, GeometryBasics.OffsetInteger{-1, UInt32}}}}}}, Wireframe{Tuple{GeometryBasics.Mesh{3, Float32, GeometryBasics.TriangleFace{Int64}, (:position,), Tuple{Vector{Point{3, Float32}}}, Vector{GeometryBasics.TriangleFace{Int64}}}}})

In [56]:
simplified_wing = simplify(angle_simplified_wing, 0.1, :adaptive)
angle_simplified_wing = simplify(simplified_wing, 0.1, :angle_optimized)
plot_with_wireframe(angle_simplified_wing)

Simplifying mesh using adaptive clustering with 10.0% reduction target
Simplification complete:
  Original vertices: 458
  Simplified vertices: 412
  Actual reduction: 10.0%
  Average triangle quality: 0.235 (0 = perfect equilateral)
Simplifying mesh using angle-optimized clustering with 10.0% reduction target
Simplification complete:
  Original vertices: 412
  Simplified vertices: 371
  Actual reduction: 10.0%
  Average triangle quality: 0.2 (0 = perfect equilateral)


(Scene (600px, 450px):
  0 Plots
  1 Child Scene:
    └ Scene (600px, 450px), LScene(), Mesh{Tuple{GeometryBasics.Mesh{3, Float32, GeometryBasics.NgonFace{3, GeometryBasics.OffsetInteger{-1, UInt32}}, (:position, :normal), Tuple{Vector{Point{3, Float32}}, Vector{Vec{3, Float32}}}, Vector{GeometryBasics.NgonFace{3, GeometryBasics.OffsetInteger{-1, UInt32}}}}}}, Wireframe{Tuple{GeometryBasics.Mesh{3, Float32, GeometryBasics.TriangleFace{Int64}, (:position,), Tuple{Vector{Point{3, Float32}}}, Vector{GeometryBasics.TriangleFace{Int64}}}}})

In [57]:
# Print max angle in angle_simplified_wing

function max_triangle_angle(mesh::GeometryBasics.Mesh)
    vertices = mesh.position
    faces = mesh.faces
    
    max_angle = 0.0
    
    for face in faces
        v1 = vertices[face[1]]
        v2 = vertices[face[2]]
        v3 = vertices[face[3]]
        
        angles = triangle_angles(v1, v2, v3)
        max_angle = max(max_angle, maximum(angles))
    end
    
    return max_angle
end

function min_triangle_angle(mesh::GeometryBasics.Mesh)
    vertices = mesh.position
    faces = mesh.faces
    
    min_angle = Inf
    
    for face in faces
        v1 = vertices[face[1]]
        v2 = vertices[face[2]]
        v3 = vertices[face[3]]
        
        angles = triangle_angles(v1, v2, v3)
        min_angle = min(min_angle, minimum(angles))
    end
    
    return min_angle
end

max_angle = max_triangle_angle(angle_simplified_wing)
min_angle = min_triangle_angle(angle_simplified_wing)

println("Max triangle angle in angle_simplified_wing: $(round(max_angle, digits=2))°")
println("Min triangle angle in angle_simplified_wing: $(round(min_angle, digits=2))°")

Max triangle angle in angle_simplified_wing: 154.39°
Min triangle angle in angle_simplified_wing: 12.5°


In [59]:
using Serialization
serialize("angle_simplified_wing.json", angle_simplified_wing)

In [63]:
deserialized_wing = deserialize("angle_simplified_wing.json")
plot_with_wireframe(deserialized_wing)

(Scene (600px, 450px):
  0 Plots
  1 Child Scene:
    └ Scene (600px, 450px), LScene(), Mesh{Tuple{GeometryBasics.Mesh{3, Float32, GeometryBasics.NgonFace{3, GeometryBasics.OffsetInteger{-1, UInt32}}, (:position, :normal), Tuple{Vector{Point{3, Float32}}, Vector{Vec{3, Float32}}}, Vector{GeometryBasics.NgonFace{3, GeometryBasics.OffsetInteger{-1, UInt32}}}}}}, Wireframe{Tuple{GeometryBasics.Mesh{3, Float32, GeometryBasics.TriangleFace{Int64}, (:position,), Tuple{Vector{Point{3, Float32}}}, Vector{GeometryBasics.TriangleFace{Int64}}}}})

In [95]:
simplified_wing = simplify(angle_simplified_wing, 0.1, :adaptive)
angle_simplified_wing = simplify(simplified_wing, 0.1, :angle_optimized)
plot_with_wireframe(angle_simplified_wing)

Simplifying mesh using adaptive clustering with 10.0% reduction target
Simplification complete:
  Original vertices: 41
  Simplified vertices: 25
  Actual reduction: 39.0%
  Average triangle quality: 0.2 (0 = perfect equilateral)
Simplifying mesh using angle-optimized clustering with 10.0% reduction target
Simplification complete:
  Original vertices: 25
  Simplified vertices: 25
  Actual reduction: 0.0%
  Average triangle quality: 0.2 (0 = perfect equilateral)


(Scene (600px, 450px):
  0 Plots
  1 Child Scene:
    └ Scene (600px, 450px), LScene(), Mesh{Tuple{GeometryBasics.Mesh{3, Float32, GeometryBasics.NgonFace{3, GeometryBasics.OffsetInteger{-1, UInt32}}, (:position, :normal), Tuple{Vector{Point{3, Float32}}, Vector{Vec{3, Float32}}}, Vector{GeometryBasics.NgonFace{3, GeometryBasics.OffsetInteger{-1, UInt32}}}}}}, Wireframe{Tuple{GeometryBasics.Mesh{3, Float32, GeometryBasics.TriangleFace{Int64}, (:position,), Tuple{Vector{Point{3, Float32}}}, Vector{GeometryBasics.TriangleFace{Int64}}}}})

In [ ]:
angle_simplified_wing


Mesh{3, Float32, GeometryBasics.TriangleFace{Int64}}
    faces: 520
    vertex position: 159


In [96]:
pl = plot_with_wireframe(angle_simplified_wing)
mesh!(pl[2], _wing_model, color=:red, alpha=0.5)

Mesh{Tuple{GeometryBasics.Mesh{3, Float32, GeometryBasics.NgonFace{3, GeometryBasics.OffsetInteger{-1, UInt32}}, (:position, :normal), Tuple{Vector{Point{3, Float32}}, Vector{Vec{3, Float32}}}, Vector{GeometryBasics.NgonFace{3, GeometryBasics.OffsetInteger{-1, UInt32}}}}}}

In [30]:
max_angle = max_triangle_angle(simplified_wing)
min_angle = min_triangle_angle(simplified_wing)

println("Max triangle angle in simplified_wing): $(round(max_angle, digits=2))°")
println("Min triangle angle in simplified_wing): $(round(min_angle, digits=2))°")

Max triangle angle in simplified_wing): 166.23°
Min triangle angle in simplified_wing): 6.51°


In [97]:
serialize("simplified_tiled_wing_v2.json", angle_simplified_wing)

In [ ]:
# # Enhanced triangle quality function with adjustable weighting
# function triangle_angle_quality(v1::Point3f, v2::Point3f, v3::Point3f, angle_weight::Float64=1.0)
#     angles = triangle_angles(v1, v2, v3)
    
#     # Calculate deviation from 60 degrees
#     deviations = abs.(angles .- 60.0)
    
#     # Apply exponential weighting to heavily penalize large deviations
#     weighted_deviations = deviations .^ (1.0 + angle_weight)
    
#     # Quality score: lower is better (0 = perfect equilateral triangle)
#     quality = sum(weighted_deviations) / (180.0^(1.0 + angle_weight))
    
#     return quality
# end

# # Alternative: Use a different penalty function that's more aggressive
# function triangle_angle_quality_aggressive(v1::Point3f, v2::Point3f, v3::Point3f, penalty_factor::Float64=5.0)
#     angles = triangle_angles(v1, v2, v3)
    
#     # Calculate deviation from 60 degrees
#     deviations = abs.(angles .- 60.0)
    
#     # Use exponential penalty for large deviations
#     penalties = exp.(deviations / 60.0 * penalty_factor) .- 1.0
    
#     # Quality score: lower is better
#     quality = sum(penalties) / (3.0 * (exp(penalty_factor) - 1.0))
    
#     return quality
# end

# # Enhanced angle-optimized clustering with stronger angle weighting
# function angle_optimized_clustering_weighted(vertices::Vector{Point3f}, faces::Vector{GeometryBasics.NgonFace{3, Int}}, target_reduction::Float64, angle_weight::Float64=3.0)
#     # Start with a copy of the original mesh
#     current_vertices = copy(vertices)
#     current_faces = copy(faces)
#     target_vertex_count = Int(round(length(vertices) * (1 - target_reduction)))
    
#     # Iteratively remove vertices with worst quality impact
#     while length(current_vertices) > target_vertex_count
#         # Find vertex pairs that can be merged
#         merge_candidates = []
        
#         for i in 1:length(current_vertices)
#             for j in (i+1):length(current_vertices)
#                 if i != j
#                     # Calculate distance between vertices
#                     dist = norm(current_vertices[i] - current_vertices[j])
                    
#                     # Only consider nearby vertices for merging
#                     merge_threshold = 0.1  # Adjustable threshold
#                     if dist < merge_threshold
#                         # Calculate quality improvement if we merge these vertices
#                         new_vertex = (current_vertices[i] + current_vertices[j]) / 2
                        
#                         # Find faces that would be affected
#                         affected_faces = [f for f in current_faces if i in f || j in f]
                        
#                         # Calculate current quality with heavy angle weighting
#                         current_quality = 0.0
#                         valid_faces = 0
#                         for face in affected_faces
#                             v1 = current_vertices[face[1]]
#                             v2 = current_vertices[face[2]]
#                             v3 = current_vertices[face[3]]
#                             current_quality += triangle_angle_quality(v1, v2, v3, angle_weight)
#                             valid_faces += 1
#                         end
                        
#                         if valid_faces > 0
#                             current_quality /= valid_faces
                            
#                             # Calculate new quality after merge
#                             new_quality = 0.0
#                             new_valid_faces = 0
                            
#                             for face in affected_faces
#                                 # Create new face with merged vertex
#                                 new_face_vertices = []
#                                 for vertex_idx in face
#                                     if vertex_idx == i || vertex_idx == j
#                                         push!(new_face_vertices, new_vertex)
#                                     else
#                                         push!(new_face_vertices, current_vertices[vertex_idx])
#                                     end
#                                 end
                                
#                                 # Skip degenerate triangles
#                                 if length(unique(new_face_vertices)) == 3
#                                     new_quality += triangle_angle_quality(new_face_vertices[1], new_face_vertices[2], new_face_vertices[3], angle_weight)
#                                     new_valid_faces += 1
#                                 end
#                             end
                            
#                             if new_valid_faces > 0
#                                 new_quality /= new_valid_faces
                                
#                                 # Heavily weight angle quality improvement
#                                 angle_improvement = current_quality - new_quality
#                                 geometric_cost = dist  # Keep geometric cost as a tie-breaker
                                
#                                 # Combined score: prioritize angle improvement over geometric cost
#                                 total_score = angle_improvement * 10.0 - geometric_cost * 0.1
                                
#                                 push!(merge_candidates, (i, j, total_score, new_vertex, angle_improvement))
#                             end
#                         end
#                     end
#                 end
#             end
#         end
        
#         # If no good merge candidates, break
#         if isempty(merge_candidates)
#             break
#         end
        
#         # Sort by total score (highest first)
#         sort!(merge_candidates, by=x->x[3], rev=true)
        
#         # Only perform merges that actually improve angle quality
#         best_candidates = filter(x -> x[5] > 0, merge_candidates)
        
#         if isempty(best_candidates)
#             println("No more beneficial merges available, stopping early.")
#             break
#         end
        
#         # Perform the best merge
#         best_merge = best_candidates[1]
#         i, j, total_score, new_vertex, angle_improvement = best_merge
        
#         # Update vertices
#         new_vertices = Point3f[]
#         vertex_mapping = Dict{Int, Int}()
        
#         for (idx, vertex) in enumerate(current_vertices)
#             if idx == i
#                 push!(new_vertices, new_vertex)
#                 vertex_mapping[idx] = length(new_vertices)
#             elseif idx == j
#                 vertex_mapping[idx] = vertex_mapping[i]
#             else
#                 push!(new_vertices, vertex)
#                 vertex_mapping[idx] = length(new_vertices)
#             end
#         end
        
#         # Update faces
#         new_faces = GeometryBasics.NgonFace{3, Int}[]
#         for face in current_faces
#             new_face_indices = [vertex_mapping[face[k]] for k in 1:3]
            
#             # Only keep non-degenerate faces
#             if length(unique(new_face_indices)) == 3
#                 push!(new_faces, GeometryBasics.NgonFace{3, Int}(new_face_indices...))
#             end
#         end
        
#         current_vertices = new_vertices
#         current_faces = new_faces
#     end
    
#     return current_vertices, current_faces
# end

# # Updated simplify function with weighted angle optimization
# function simplify(mesh::GeometryBasics.Mesh, intensity::Float64, algorithm::Symbol, angle_weight::Float64=3.0)
#     # Extract vertices and faces from input mesh
#     vertices = mesh.position
#     faces = [GeometryBasics.NgonFace{3, Int}([Int(f[i].i) + 1 for i in 1:3]...) for f in mesh.faces]
    
#     # Validate algorithm parameter
#     if algorithm ∉ [:simple, :adaptive, :angle_optimized, :angle_weighted]
#         throw(ArgumentError("Algorithm must be :simple, :adaptive, :angle_optimized, or :angle_weighted, got :$algorithm"))
#     end
    
#     # Validate intensity parameter
#     if intensity < 0.0 || intensity > 1.0
#         throw(ArgumentError("Intensity must be between 0.0 and 1.0, got $intensity"))
#     end
    
#     # Apply the appropriate simplification algorithm
#     if algorithm == :simple
#         grid_size = 0.01 + intensity * 0.99
#         println("Simplifying mesh using vertex clustering with grid size: $grid_size")
#         simplified_vertices, simplified_faces = vertex_clustering_simplification(vertices, faces, grid_size)
        
#     elseif algorithm == :adaptive
#         target_reduction = intensity
#         println("Simplifying mesh using adaptive clustering with $(target_reduction*100)% reduction target")
#         simplified_vertices, simplified_faces = adaptive_vertex_clustering(vertices, faces, target_reduction)
        
#     elseif algorithm == :angle_optimized
#         target_reduction = intensity
#         println("Simplifying mesh using angle-optimized clustering with $(target_reduction*100)% reduction target")
#         simplified_vertices, simplified_faces = angle_optimized_clustering(vertices, faces, target_reduction)
        
#     elseif algorithm == :angle_weighted
#         target_reduction = intensity
#         println("Simplifying mesh using weighted angle-optimized clustering with $(target_reduction*100)% reduction target (angle weight: $angle_weight)")
#         simplified_vertices, simplified_faces = angle_optimized_clustering_weighted(vertices, faces, target_reduction, angle_weight)
#     end
    
#     # Create and return the simplified mesh
#     simplified_mesh = GeometryBasics.Mesh(simplified_vertices, simplified_faces)
    
#     # Print simplification results
#     original_vertex_count = length(vertices)
#     simplified_vertex_count = length(simplified_vertices)
#     actual_reduction = round((1 - simplified_vertex_count / original_vertex_count) * 100, digits=1)
    
#     println("Simplification complete:")
#     println("  Original vertices: $original_vertex_count")
#     println("  Simplified vertices: $simplified_vertex_count")
#     println("  Actual reduction: $(actual_reduction)%")
    
#     # Calculate and report triangle quality
#     if !isempty(simplified_faces)
#         total_quality = 0.0
#         for face in simplified_faces
#             v1 = simplified_vertices[face[1]]
#             v2 = simplified_vertices[face[2]]
#             v3 = simplified_vertices[face[3]]
#             total_quality += triangle_angle_quality(v1, v2, v3, 1.0)  # Use standard weighting for reporting
#         end
#         avg_quality = total_quality / length(simplified_faces)
#         println("  Average triangle quality: $(round(avg_quality, digits=3)) (0 = perfect equilateral)")
#     end
    
#     return simplified_mesh
# end

# # Demo function to compare different angle weights
# function demo_angle_weighting()
#     # Create a test mesh
#     S = GeometryBasics.mesh(GeometryBasics.Sphere(Point3f(0.0, 0.0, 0.0), 1.0))
    
#     println("=== Comparing Different Angle Weights ===")
#     println("Original mesh: $(length(S.position)) vertices, $(length(S.faces)) faces")
    
#     # Test different angle weights
#     weights = [1.0, 3.0, 5.0, 10.0, 50.0]
    
#     println("\n--- Original Mesh Quality ---")
#     analyze_triangle_quality(S)
    
#     results = []
    
#     for weight in weights
#         println("\n--- Angle Weight: $weight ---")
#         result = simplify(S, 0.5, :angle_weighted, weight)
#         analyze_triangle_quality(result)
#         push!(results, result)
#     end
    
#     # Visualize results
#     fig = Figure(size = (1200, 800))
    
#     # Original
#     ax1 = Axis3(fig[1, 1], title="Original\n$(length(S.position)) vertices")
#     mesh!(ax1, S, color=:blue, alpha=0.8)
    
#     # Different weights
#     for (i, (weight, result)) in enumerate(zip(weights, results))
#         row = (i - 1) ÷ 2 + 1
#         col = (i - 1) % 2 + 2
#         ax = Axis3(fig[row, col], title="Weight: $weight\n$(length(result.position)) vertices")
#         mesh!(ax, result, color=:purple, alpha=0.8)
#     end
    
#     display(fig)
    
#     return results
# end

demo_angle_weighting (generic function with 1 method)